Here are some more side tests to clarify / justify details, that would clutter the main test notebook.

In [ ]:
# from importlib import reload
import helper as hlp

import numpy as np
import numpy.lib.recfunctions

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mpldates
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm, Normalize
%matplotlib inline

import scipy.interpolate as sci
import scipy.optimize as sco
import scipy.integrate as scint
import scipy.stats as scs
from scipy.stats import rv_continuous
import scipy.signal as scsignal

from tqdm import tqdm_notebook as tqdm
import json
import datetime
import pickle
from astropy.time import Time as astrotime
from corner import corner

import sklearn.neighbors as skn
import sklearn.model_selection as skms

# from corner_hist import corner_hist
from anapymods3.plots import dg, split_axis, get_binmids, hist_marginalize
from anapymods3.stats import json2kde, GaussianKDE, rejection_sampling
from anapymods3.general.misc import fill_dict_defaults

import tdepps.bg_injector as BGInj
import tdepps.bg_rate_injector as BGRateInj
import tdepps.rate_function as RateFunc
import tdepps.llh as LLH
import tdepps.analysis as Analysis
import tdepps.signal_injector as SigInj

secinday = 24. * 60. * 60.

# Load data

Load IC86 data from epinat, which should be the usual IC86-I (2011) PS sample, but pull corrected and OneWeights corrected by number of events generated.

In [ ]:
exp = np.load("data/IC86_I_data.npy")
mc = np.load("data/IC86_I_mc.npy")
# Use the officially stated livetime, not the ones from below
livetime = 332.61

# Data livetime comparison to v1.4

Let's compare to the v1.4 list, as used by jfeintzig.
Oddly we have 0.2 days less livetime as he had.
The number of runs is correct though

In [ ]:
goodrun_dict, ic_livetime = hlp.create_goodrun_dict(
    runlist="data/runlists/ic86-i-goodrunlist.json", filter_runs=None)
nruns = len(goodrun_dict["run"])

# Make a recarray to use older code below
types = [v.dtype for v in goodrun_dict.values()]
dtype = [(str(n), f) for n, f in zip(goodrun_dict.keys(), types)]
inc_run_arr = np.empty((nruns,), dtype=dtype)

for k, v in goodrun_dict.items():
    inc_run_arr[k] = v
    
print(inc_run_arr.dtype.names)

In [ ]:
# New livetime from iclive runlist
print("Total runs from iclive     : ", len(inc_run_arr["run"]))
print("IC86-I livetime from iclive: ", ic_livetime)

In [ ]:
# For comparison, also parse the v1.4 list
# Should be: 1081 runs, with a total livetime of 332.61 days.
with open("data/runlists/Prelim_IC86-I_v1.4a.txt",'r') as f:
    data = []
    for line in f.readlines():
        data.append(line.replace('\n',''))
        
# Skip to beginning of run info
data = data[73:]

# Split at white space
data = [d.split() for d in data]

dtype = [("runID", np.int), ("duration", np.float), ("IT", "|S2"),
         ("CONF", "|S7"), ("FLAG", "|S6")]
runlist = np.empty((len(data),), dtype=dtype)

runlist["runID"] = np.array([int(d[0]) for d in data])
runlist["duration"] = np.array([float(d[3]) for d in data])
runlist["IT"] = np.array([d[5] for d in data])
runlist["CONF"] = np.array([d[6] for d in data])
runlist["FLAG"] = np.array([d[7] for d in data])

# Now filter: Include IT=it, CONF=full, FLAG=GOOD, exclude strange rate runs
exclude_rate = [120028, 120029, 120030, 120087, 120156, 120157]
itgood = runlist["IT"] == b"IT"  # Somehow only bitwise comparison is non-empty
confgood = runlist["CONF"] == b"full"
flaggood = runlist["FLAG"] == b"GOOD"
ratebad = np.in1d(runlist["runID"], exclude_rate)

include = itgood & confgood & flaggood & ~ratebad
runlist_inc = runlist[include]

# Get the livetime of the sample in days
hoursindays = 24.
secinday = hoursindays * 60. * 60.
old_livetime = np.sum(runlist_inc["duration"]) / hoursindays

print("Total runs from v1.4     : ", len(runlist_inc))
print("Total livetime from v1.4 : ", old_livetime)

Let's see, if the 120 extra runs in the new runlist make up for the difference of about 10 days in livetime.

In [ ]:
iclive_in_old = np.in1d(inc_run_arr["run"], runlist_inc["runID"])
not_in_old = inc_run_arr[~iclive_in_old]

start = not_in_old["good_start_mjd"]
stop = not_in_old["good_stop_mjd"]
missing_livetime = np.sum(stop - start)

print("\nOfficial IC86-I PS livetime: ", livetime)
print("Total livetime from v1.4   : ", old_livetime)
print("IC86-I livetime from iclive: ", ic_livetime)

print("\nMissing runs in old: ", len(not_in_old))
print("Livetime icliv - old :", ic_livetime - old_livetime)

print("\nDiff from summing missing runs           : ", missing_livetime)
print("New iclive livetime with same runs as old: ",
      ic_livetime - missing_livetime)

print("\nTotal rate [Hz] over total livetime: ",
      len(exp) / (livetime * secinday))

All runs from the new run list that have zero events, make up for the missing runs in the old runlist, so this is consisting.

Dont't know though, where the missing 0,2 days come from. Probably some runtimes have shifted a little making some extra livetime in the new list.

In [ ]:
# Store events in bins with run borders
exp_times = exp["timeMJD"]
start_mjd = inc_run_arr["good_start_mjd"]
stop_mjd = inc_run_arr["good_stop_mjd"]

tot = 0
evts_in_run = {}
for start, stop , runid in zip(start_mjd, stop_mjd, inc_run_arr["run"]):
    mask = (exp_times >= start) & (exp_times < stop)
    evts_in_run[runid] = exp[mask]
    tot += np.sum(mask)
    
# Crosscheck, if we got all events and counted nothing double
print("Do we have all events? ", tot == len(exp))
print("  Events selected : ", tot)
print("  Events in exp   : ", len(exp))

# Create binmids and histogram values in each bin
binmids = 0.5 * (start_mjd + stop_mjd)
h = np.zeros(len(binmids), dtype=np.float)

for i, evts in enumerate(evts_in_run.values()):
    h[i] = len(evts)
    
m = (h > 0)
print("Runs with 0 events :", np.sum(~m))
print("Runtime in those runs: ", np.sum(inc_run_arr["good_stop_mjd"][~m] -
                                        inc_run_arr["good_start_mjd"][~m]))

# Remove all zero event runs (artifacts from new run list) and calc the rate
stop_mjd, start_mjd = stop_mjd[m], start_mjd[m]
h = h[m] / ((stop_mjd - start_mjd) * secinday)
binmids = binmids[m]

# Time dependent rate function

**Note: I think it is unnecessary to use a time and declination dependent rate. The spatial part is injected from the data BG from KDE anyways. So we just need to have the rate to determine how much events we inject allsky.**

Rate ist time dependent because of seasonal variation.
We take this varariation into account by fitting a priodic function to the time resolved rate.

The data is built by calculating the rate in each run as seen before.
This rate is correctly normalized and smoothes local fluctuations.

### Peridoc function with a weighted least squares fit

See side_test for comparison to spline fits.
The function is a simple sinus scalable by 4 parameters to fit the shape of the rates:

$$
    f(x) = a\cdot \sin(b\cdot(x - c)) + d
$$

The least squares loss function is

$$
    R = \sum_i (w_i(y_i - f(x_i)))^2
$$

Weights are standard deviations from poisson histogram error.

$$
    w_i = \frac{1}{\sigma_i}
$$

Seed values are estimated from plot rate vs time.

- Period should be 365 days (MJD) because we have one year of data so we choose $b0 = 2\pi/365$.
- Amplitude is about $a_0=-0.0005$, because sinus seems to start with negative values.
- The x-offset is choose as the first start date, to get the right order of magnitude.
- The y-axis intersection $d$ schould be close to the weighted average, so we take this as a seed.

The bounds are motivated as follows (and if we don't hit them, it's OK to use them).

- Amplitude $a$ should be positive, this also resolves a degenracy between a-axis offset.
- The period $b$ should scatter around one year, a period larger than +-1 half a year is unphysical.
- The x-offset $c$ cannot be greater than the initial +- the period because we have a periodic function.
- The y-axis offset $d$ is arbitrarily constrained, but as seen from the plot it should not exceed 0.1. 

## Proposed was something like this

Rate ist time dependent because of seasonal variation and delination dependent because the detector acceptance is declination dependent.
A correletation should not exist or be very small.

So we express the rate in depence of time and decliantion as

$$
    R(t,\delta) = R_T(t)\cdot R_\delta(t)
$$

with independent parts in declination and time each.

The time parts is constructed by fitting a periodic function.
Then we seperatly fit a spline to the total $\sin(\delta)$ distribtuion and normalizes it over the declination range of the sample.
For a given time and declination the smooth function $R(t, \delta)$ gives the correct detector rate.

## Splinefit to sinDec

First we fit a spline to the sinDec distribtuion for all events.
This is equivalent to what is done in skylab to estimate the per signal background PDF from data.

In [ ]:
# This is equivalent to skylab's baclground PDF construction
sinDec_bins = 25
sinDec_range= [-1, 1]
sinDec_hist, sinDec_bins = np.histogram(exp["sinDec"], density=True,
                                        bins=sinDec_bins, range=sinDec_range)

m = get_binmids([sinDec_bins])[0]

if np.any(sinDec_hist <= 0.):
    raise ValueError(("Declination hist bins empty, this must not happen. "
                      +"Empty bin idx: {}".format(
                          np.arange(len(m))[sinDec_hist <= 0.])))

# Fit to logarithm, to avoid ringing. Raise err if evaluated outside range
sinDec_spline = sci.InterpolatedUnivariateSpline(m, np.log(sinDec_hist),
                                                 k=3, ext="extrapolate")

# Normalize to area on whole sky = 1, so norm = 2pi * integral(exp(spl))
def sinDec_pdf_(x):
    return np.exp(sinDec_spline(x))

norm = scint.quad(sinDec_pdf_, -1, 1)[0] * 2. * np.pi

def sinDec_pdf(x):
    return (np.exp(sinDec_spline(x))) / norm

print("SinDec pdf has area on 4pi = ", scint.quad(
    sinDec_pdf, -1, 1)[0] * 2 * np.pi)

In [ ]:
x = np.linspace(sinDec_bins[0], sinDec_bins[-1], 100)
y = sinDec_pdf(x)

plt.plot(x, y)

In [ ]:
# Try to hist dec dependence of single run
# As expected we have not enough statistic to see anything
run = 60  # OFten these lead to empty hists...
start = start_mjd[run]
stop = stop_mjd[run]
mask = (exp_times >= start) & (exp_times < stop)
_sinDec_run = np.sin(exp["dec"][mask])

plt.hist(_sinDec_run, range=[-1, 1], bins=10)

## Spline to rate distribution

Choosing spline weights according to scipy.interpolate.UnivariateSpline manual:

    If None (default), s = len(w) which should be a good value if 1/w[i] is an estimate of
    the standard deviation of y[i].

Internally it is doing a weighted least squares fit with $\sum_i(w_i(y_i-\text{spl}(x_i)))^2 \leq s$.
We leave $s$ as the default because we have an estimate for the stddevs: $\sigma_i = \sqrt{h_i}$.
To match the definition of the weights we use:

$$
    \frac{1}{w_i} \stackrel{!}{=} \sigma_i = \sqrt{h_i} \Leftrightarrow w_i 
                               = \frac{1}{\sqrt{h_i}}
$$

Because we scaled $h_i$ to get the rate in events per s, we need to scale the errors too:

$$
    \tilde{h}_i = \frac{h_i}{s_i} \Rightarrow \tilde{\sigma}_i = \frac{\sqrt{h_i}}{s_i} 
                = \frac{\sqrt{s_i\tilde{h}_i}}{s} 
                = \sqrt{\frac{\tilde{h}_i}{s_i}} = \frac{1}{w_i}
$$

The smoothing condition `s` chooses the support knots based on the weights.
Because we have some oscilating pattern due to to seasonal variations (periode ~1yr) a quadratic spline function is not enough.
So we choose the next higher order, a cubic spline, which is able to oscilate up and down exatcly once.

**Note:** If a weight is zero, the corresponding point doesn't contribute at all.
So we might consider using $w_i = \sigma_i$ instead.
Then point woth high poisson statsitics are preferred over low statistic bins.
It doesn't seem to make a huge difference though.

Below we try both weights and the unweighted case.
For the 'correctly' weighted case with $w_i = 1. / \sigma_i$ the spline oscillates strongly.
So we better try a true perdiodic function.

In [ ]:
# h is already scaled, so we need to scale the errors too
yerr = np.sqrt(h) / np.sqrt((stop_mjd - start_mjd) * secinday)
# Originally used w = 1. / yerr, but for poisson bins use rel. error
w = h / yerr
rate_spline = sci.UnivariateSpline(binmids, h, k=3, w=w,
                                   s=None, ext="extrapolate")

rate_spline_inv = sci.UnivariateSpline(binmids, h, k=3, w=yerr,
                                       s=None, ext="extrapolate")

rate_spline_unw = sci.UnivariateSpline(binmids, h, k=3, w=None,
                                       s=None, ext="extrapolate")

In [ ]:
# Plot runs
xerr = 0.5 * (stop_mjd - start_mjd)
plt.errorbar(binmids, h, xerr=0, yerr=yerr, fmt=",")
plt.ylim(0, None);

# Plot spline
x = np.linspace(start_mjd[0], stop_mjd[-1], 200)
y = rate_spline(x)
plt.plot(x, y, zorder=5, lw=2, color="k", label="w=h/err")

# Plot weighted average. Weights are variance to resemble stddev weighted
# least squares fit
avg = np.average(h, weights=yerr**2)
plt.axhline(avg, 0, 1, color="k", ls="--", zorder=5)

# Plot unweighted mean and spline
y = rate_spline_unw(x)
plt.plot(x, y, zorder=5, lw=2, color="r")

avg = np.mean(h)
plt.axhline(avg, 0, 1, color="r", ls="--", zorder=5, label="unweighted")

# Plot with inverse weights
y = rate_spline_inv(x)
plt.plot(x, y, zorder=5, lw=2, color="g", label="w=err")
avg = np.average(h, weights=1. / yerr**2)
plt.axhline(avg, 0, 1, color="g", ls="--", zorder=5)

plt.xlim(start_mjd[0], stop_mjd[-1])
plt.legend()
plt.tight_layout()
# plt.savefig("./data/figs/time_rate_splines.png", dpi=200)
plt.show()

## Periodic function fit

Try a peridoc function with a weighted least squares fit.

$$
    f(x) = a\cdot \sin(b\cdot(x - c)) + d
$$

The least squares loss function is

$$
    R = \sum_i (w_i(y_i - f(x_i)))^2
$$

Weights are standard deviations from poisson histogram error.

$$
    w_i = \frac{1}{\sigma_i}
$$

Seed values are estimated from plot rate vs time.
Period should be 365 days (MJD) because we have one year of data so we choose $b0 = 2\pi/365$.
Amplitude is about $a_0=-0.0005$, because sinus seems to start with negative values.
The x-offset is choose as the first start date, to get the right order of magnitude.
The y-axis intersection $d$ schould be close to the weighted average, so we take this as a seed.

The bounds are motivated as follows (and if we don't hit them, it's OK to use them).
Amplitude $a$ should be positive, this also resolves a degenracy between a-axis offset.
The period $b$ should scatter around one year, a period larger than +-1 half a year is unphysical.
The x-offset $c$ cannot be greater than the initial +- the period because we have a periodic function.
The y-axis offset $d$ is arbitrarily constrained, but as seen from the plot it should not exceed 0.1. 

In [ ]:
def f(x, args):
    a, b, c, d = args
    return a * np.sin(b * (x - c)) + d

def lstsq(pars, *args):
    """
    Weighted leastsquares min sum((wi * (yi - fi))**2)
    """
    # data x,y-values and weights are fixed
    x, y, w = args[0], args[1], args[2]
    # Params get fitted
    a, b, c, d = pars[0], pars[1], pars[2], pars[3]
    # Target function
    f = a * np.sin(b * (x - c)) + d
    # Least squares loss
    return np.sum((w * (y - f))**2)

In [ ]:
# Seed values from consideration above.
a0 = -0.0005
b0 = 2. * np.pi / 365.
c0 = np.amin(start_mjd)
d0 = np.average(h, weights=yerr**2)

x0 = [a0, b0, c0, d0]
# Bounds as explained above
bounds = [[None, None], [0.5 * b0, 1.5 * b0], [c0 - b0, c0 + b0, ], [0, 0.01]]
# x, y values, weights
w = h / yerr  # Originally used 1 / yerr
args = (binmids, h, w)

res = sco.minimize(fun=lstsq, x0=x0, args=args, bounds=bounds)

for i, name in enumerate(["Amplitude a", "Period b", "x-Shift c", "y-axis d"]):
    print(name, " : ", res.x[i])

In [ ]:
# Plot runs
xerr = 0.5 * (stop_mjd - start_mjd)
plt.errorbar(binmids, h, xerr=0, yerr=yerr, fmt=",")
plt.ylim(0, None);

# Plot fit
pars = res.x
x = np.linspace(start_mjd[0], stop_mjd[-1], 1000)
y = f(x, pars)
plt.plot(x, y, zorder=5)

# Plot y shift dashed to see baseline or years average
plt.axhline(res.x[3], 0, 1, color="C1", ls="--")

plt.xlim(start_mjd[0], stop_mjd[-1])
plt.tight_layout()

# plt.savefig("./data/figs/time_rate_sinus.png", dpi=200)

## Combine both to make a time-dec rate function

Multiply the rate function of time with the pdf in sinDec.
This gives the rate per solid angle.
Integrated over the whole sphere, we recover the total rate that time.
Integrating further over the whole time range, regarding the deadtimes of the detector, we recover the number of total events in all runs in this sample.
We can approximate this by using the fitted y-axis offset, which is approximatly the mean and multiply with the livetitme.
We recover the number of total events to good approximation.

In [ ]:
print("Number of events from approx : ", res.x[3] * livetime * secinday)
print("True number of events        : ", len(exp))

# Simply integrating doesn't respect the downtimes
wrong = scint.quad(f, start_mjd[0], stop_mjd[-1], args=res.x)[0] * secinday
print("Integrating over whole year  : ", wrong)

In [ ]:
# Function of time, sinDec and right-ascension to get the rate at that point.
def time_sinDec_rate(sinDec, t):
    return sinDec_pdf(sinDec) * f(t, res.x)

In [ ]:
try:
    print(sinDec_pdf)
except NameError:
    raise NameError("Execute 'Splinefit to sinDec' cell to get needed functions")

# This should yield ~1. The ratio of the fitted average d and the integral
# of the rate function over the whole sky at a time approximately at rate=d
_i = 2. * np.pi * scint.quad(time_sinDec_rate, -1, 1, args=55700)[0] / res.x[3]
print("1D and mukltiply by 2pi : ",_i)

# We can also use a 2D integrator to integrate RA as well (same result)
def fullsky_rate(ra, sinDec, t):
    return sinDec_pdf(sinDec) * f(t, res.x)
_i = scint.dblquad(fullsky_rate, -1, 1, lambda x: 0, lambda x: 2.*np.pi,
                   args=(55700,))[0] / res.x[3]
print("2D over dec and ra      : ", _i)

# Sampling the number of BG events to inject

Testing if the integration of the rate function gives the correct number of events to inject.
For small time windows it should be the same as just taking the rate times the window size (rectangular approximation).

In [ ]:
def rate_integral(trange, pars):
        """
        Match with factor [secinday] = 24 * 60 * 60 s / MJD = 86400/(Hz*MJD)
        in the last step.
            [a], [d] = Hz, [b], [c], [ti] = MJD
            [a / b] = Hz * MJD, [d * (t1 - t0)] = HZ * MJD
        """
        a, b, c, d = pars
        
        t0 = np.atleast_2d(trange[:, 0]).reshape(len(trange), 1)
        t1 = np.atleast_2d(trange[:, 1]).reshape(len(trange), 1)
        
        per = a / b * (np.cos(b * (t0 - c)) - np.cos(b * (t1 - c)))
        lin = d * (t1 - t0)

        return (per + lin) * secinday


def get_num_of_bg_events(t, trange, ntrials, pars):
    """
    Draw number of background events per trial from a poisson distribution
    with the mean of the fitted rate function.
    Then draw nevents times via rejection sampling for the time dpeendent rate
    function.
    
    Parameters
    ----------
    t : array-like
        Times of the occurance of each source event in MJD.
    trange : [float, float] or array_like, shape (len(t), 2)
        Time window(s) in seconds relativ to the given time(s) t.
        - If [float, float], the same window [lower, upper] is used for every
          source.
        - If array-like, lower [i, 0] and upper [i, 1] bounds of the time
          window per source.
    ntrials : int
        Number of background trials we need the number of how many events to
        inject for.
    pars : array-like
        Best fit parameters from the fit function used in its integral.
        
    Returns
    -------
    sample : list, length len(t)
        Contains a dict with keys "times" and "nevents"
    times : 
    nevents : array-like, shape (len(t), ntrials)
        The number of events to inject for each trial for each source.
    """
    t = np.array(t)
    trange = np.array(trange)
    nsrc = len(t)
    
    # Make shape (nsources, 1) for the times
    t = t.reshape(nsrc, 1)
    
    # If range is 1D (one for all) reshape it to (nsources, 2)
    if len(trange.shape) == 1:
        print("Using the same time window for all sources.")
        trange = np.repeat(trange.reshape(1, 2), repeats=nsrc, axis=0)
        
    # Prepare time window in MJD
    trange = t + trange / secinday
    
    # Expectation is the integral in the time frame
    expect = rate_integral(trange, pars)
        
    # Sample from poisson
    nevts = np.random.poisson(lam=expect, size=(nsrc, ntrials))
    
    return nevts

In [ ]:
t = start_mjd[100:104]
trange = np.array([-120, 220])
ntrials = 10

nevts = get_num_of_bg_events(t=t, trange=trange, ntrials=ntrials, pars=res.x)
print("Events to inject:\n", nevts)

# Test if integral and simple approximation is the same for small time frames
t = np.array(t)
trange = np.array(trange)
t = t.reshape(len(t), 1)
trange = np.repeat(trange.reshape(1, 2), repeats=len(t), axis=0)
trange = t + trange / secinday
rate = f(t, res.x)
nevts = rate_integral(trange, res.x)
print("\nFor small windows, rate * dt should be equal to integral")
print("Time window dt in seconds:\n", np.diff(trange))
print("Rate * dt:\n", rate * np.diff(trange, axis=1) * secinday)
print("Integral:\n", nevts)
print(np.allclose(rate * np.diff(trange, axis=1) * secinday, nevts))

# Time PDF ratio

Background in uniformly distributed in the time window.
Signal distribtution is falling off gaussian-like at both edges so normalization is different.
So the ratio S/B is simply the the signal pdf divided by the uniform normalization $1 / (t_1 - t_0)$ in the time frame.

To get finite support we truncate the gaussian edges at n sigma.
Though arbitrarliy introducet to smoothly run to zero, the concrete cutoff of the doesn't really matter (so say 4, 5, 6 sigma, etc).
This is because in the LLH we get the product of $\langle b_B \rangle B_i$.
A larger cutoff make the normalization of the BG pdf larger, but in the same time makes the number of expected BG event get higher in the same linear fashion.
So as long as we choose a cutoff which ensures that $S \approx 0$ outside, we're good to go.

In [ ]:
secinday = 24. * 60. * 60.

def time_bg_pdf(t, t0, a, b):
    """
    BG is uniform for t in [t0 + a, t0 + b] and 0 outside.
    
    Times t and t0 are given in MJD, the range is given relative to t0
    in seconds. t are the times we return pdf values for, t0 is the time of
    the source event around which the time frame is defined.
    
    The PDF is normed to time in seconds!
    """
    # Normalize relative to t0 in seconds (first multiply avoids rounding?)
    _t = t * secinday - t0 * secinday
  
    pdf = np.zeros_like(_t, dtype=np.float)
    uni = (_t >= a) & (_t <= b)
    pdf[uni] = 1. / (b - a)
    return pdf

def time_sig_pdf(t, t0, dt, nsig=4):
    """
    Signal falls of with gaussian with sigma = dt outside uniform range dt.
    
    Times t, t0 are in MJD, dt is in seconds.
    t are the times we return pdf values for, t0 is the time of the source
    event around which the time frame is defined.
    dt is the time window starting from t0 in which signal is uniform.
    
    The PDF is normed to time in seconds!
    """
    if dt < 0:
        raise ValueError("dt must not be negative.")

    # Normalize relative to t0 in seconds (first multiply avoids rounding?)
    _t = t * secinday - t0 * secinday
    
    # Constrain sig_t to [2, 30]s regardless of uniform time window
    sig_t = np.clip(dt, 2, 30)
    sig_t_clip = nsig * sig_t
    gaus_norm = (np.sqrt(2 * np.pi) * sig_t)
    
    # Split in def regions gaus rising, uniform, gaus falling and zero
    gr = (_t < 0) & (_t >= -sig_t_clip)
    gf = (_t > dt) & (_t <= dt + sig_t_clip)
    uni = (_t >= 0) & (_t <= dt)
    
    pdf = np.zeros_like(t, dtype=np.float)
    pdf[gr] = scs.norm.pdf(_t[gr], loc=0, scale=sig_t)
    pdf[gf] = scs.norm.pdf(_t[gf], loc=dt, scale=sig_t)
    # Connect smoothly with the gaussians
    pdf[uni] = 1. / gaus_norm
    
    # Normalize whole distribtuion
    dcdf = (scs.norm.cdf(dt + sig_t_clip, loc=dt, scale=sig_t) -
            scs.norm.cdf(-sig_t_clip, loc=0., scale=sig_t))
    norm = dcdf + dt / gaus_norm
    
    return pdf / norm

def time_soverb(t, t0, dt, nsig):
    """
    Time signal over background PDF.
    
    Signal and background PDFs are each normalized over seconds.
    Signal PDF has gaussian edges to smoothly let it fall of to zero, the
    stddev is dt when dt is in [2, 30]s, otherwise the nearest edge.

    To ensure finite support, the edges are truncated after nsig * dt.

    Parameters
    ----------
    t : array-like
        Times given in MJD for which we want to evaluate the ratio.
    t0 : float
        Time of the source event.
    dt : float
        Time window in seconds starting from t0 in which the signal pdf is
        assumed to be uniform. Must not be negative.
    nsig : float
        Clip the gaussian edges at nsig * dt
    """
    if dt < 0:
        raise ValueError("dt must not be negative.")

    secinday = 24. * 60. * 60.

    # Normalize relative to t0 in seconds (first multiply avoids rounding?)
    _t = t * secinday - t0 * secinday
   
    # Create signal PDF
    # Constrain sig_t to [2, 30]s regardless of uniform time window
    sig_t = np.clip(dt, 2, 30)
    sig_t_clip = nsig * sig_t
    gaus_norm = (np.sqrt(2 * np.pi) * sig_t)
    
    # Split in def regions gaus rising, uniform, gaus falling
    gr = (_t < 0) & (_t >= -sig_t_clip)
    gf = (_t > dt) & (_t <= dt + sig_t_clip)
    uni = (_t >= 0) & (_t <= dt)
    
    pdf = np.zeros_like(t, dtype=np.float)
    pdf[gr] = scs.norm.pdf(_t[gr], loc=0, scale=sig_t)
    pdf[gf] = scs.norm.pdf(_t[gf], loc=dt, scale=sig_t)
    # Connect smoothly with the gaussians
    pdf[uni] = 1. / gaus_norm
    
    # Normalize signal distribtuion
    dcdf = (scs.norm.cdf(dt + sig_t_clip, loc=dt, scale=sig_t) -
            scs.norm.cdf(-sig_t_clip, loc=0., scale=sig_t))
    norm = dcdf + dt / gaus_norm
    pdf /= norm
    
    # Calculate the ratio
    bg_pdf = 1. / (dt + 2 * sig_t_clip)
    ratio = pdf / bg_pdf
    return ratio

In [ ]:
# Plot a the signal and BG PDFs for a single case
# Arbitrary start date from data
t0 = start_mjd[100]
t0_sec = t0 * secinday

# dt from t0 in seconds, clip at 4 sigma
dt = 10
nsig = 4.

# Make t values for plotting in MJD around t0
clip = np.clip(dt, 2, 30) * nsig
plt_rng = [-clip, dt + clip]
t = np.linspace(t0_sec + plt_rng[0], t0_sec + plt_rng[1], 200) / secinday

bg_pdf = time_bg_pdf(t, t0, -clip, dt + clip)
sig_pdf = time_sig_pdf(t, t0, dt, nsig)

# Plot in normalized time
_t = t * secinday - t0 * secinday
plt.plot(_t, bg_pdf, "C0-")
plt.plot(_t, sig_pdf, "C1-")
plt.axvline(dt, 0, 1, color="C7", ls="--")
plt.axvline(0, 0, 1, color="C1", ls="--")

plt.xlabel("Time relative to t0 in sec")
plt.ylim(0, None);
plt.show()

# Integrate both pdf over time range to show they are correctly normalized
# Note that PDFs are defined in second so we multiply by secinday 
bg_int = scint.quad(time_bg_pdf, t[0], t[-1],
                    args=(t0, -clip, dt + clip))[0] * secinday
sig_int = scint.quad(time_sig_pdf, t[0], t[-1],
                    args=(t0, dt, nsig))[0] * secinday

print("BG integral     : ", bg_int)
print("Signal integral : ", sig_int)

In [ ]:
# Make a plot with ratios for different time windows as in the paper
# Arbitrary start date from data
t0 = start_mjd[100]
t0_sec = t0 * secinday

# dt from t0 in seconds, clip at 4 sigma
dts = [5, 50, 200]
nsig = 4

# Make t values for plotting in MJD around t0, fitting all in one plot
max_dt = np.amax(dts)
clip = np.clip(max_dt, 2, 30) * nsig
plt_rng = np.array([-clip, max_dt + clip])
t = np.linspace(t0_sec + 1.2 *plt_rng[0],
                t0_sec + 1.2 * plt_rng[1], 1000) / secinday
_t = t * secinday - t0 * secinday

# Mark event time
plt.axvline(0, 0, 1, c="#353132", ls="--", lw=2)

colors = ["C0", "C3", "C2"]
for i, dt in enumerate(dts):
    # Plot ratio S/B
    SoB = time_soverb(t, t0, dt, nsig)
    plt.plot(_t, SoB, lw=2, c=colors[i],
             label=r"$T_\mathrm{{uni}}$: {:>3d}s".format(dt))
    # Fill uniform part, might look nicely
    # fbtw = (_t > 0) & (_t < dt)
    # plt.fill_between(_t[fbtw], 0, SoB[fbtw], color="C7", alpha=0.1)

# Make it look like the paper plot, but with slightly extended borders, to
# nothing breaks outside the total time frame
plt.xlim(1.2 * plt_rng)
plt.ylim(0, 3)
plt.xlabel("t - t0 in sec")
plt.ylabel("S / B")
plt.legend(loc="upper right")
plt.grid()
plt.show()

# Declination bump in data a south

At declination -pi/2 there are a lot of events, that show up as a large spike in sin(dec).
Where does this come from?
Those events come directly from above (southern sky, zenith=0)

In [ ]:
bins = 20
ev_dec = exp["dec"]
sin_dec = np.sin(ev_dec)

fig, (axl, axr) = plt.subplots(1, 2, figsize=(12, 4))
_ = axl.hist(sin_dec, bins=200, normed=False)
_ = axr.hist(np.rad2deg(ev_dec), bins=500, normed=False)

axr.set_xlabel("sinDec")
axr.set_xlabel("Dec in °")

axl.arrow(x=-1, y=1000, dx=0, dy=-100, head_length=20, head_width=0.03, lw=2,
          fc="C1", ec="C1")
axr.arrow(x=-90, y=300, dx=0, dy=-200, head_length=20, head_width=3, lw=2,
          fc="C1", ec="C1")

fig.tight_layout()

# Integrate Kent function in 2D

The paraboloid sigma was gained from estimating a circular region, in which the 2D llh includes 39% of the total probability (Attention: chi2(1, df=2), because 2D gaussian behave differently).
This assumes a gaussian llh function, so when using the usual gaussian in the signal pdf we can directly use this sigma estimate for the gaussian sigma.

For a Kent distribtuions kappa however, it is unclear how both are connected.
For starters, when kappa is 0, points are uniform on the sky, but fully concentrated in a gaussian and vice versa.
We could simply assume $\kappa = 1/\sigma$ but this is mostly founded on the 1D Mises distribution which has this property for small sigma only.

So here we try the following:

- For a fixed sigma, try to find the corresponding kappa, so that when integrating the Kent PDF with this kappa in a circle with radius sigma, the probability content in that circle is 68%.

The Kent PDF is given by:

$$
    f(\vec{x}_i|\vec{x}_S) = \frac{\kappa}{4\pi\sinh{\kappa}}\cdot\exp(\kappa(\vec{x}_i\cdot\vec{x}_S))
                           = \frac{\kappa}{4\pi\sinh{\kappa}}\cdot\exp(\kappa\cos\Psi)
$$

where $\cos\Psi$ is the angular distance of both vectors.

Because we need to integrate over the surface of the unit sphere, we define the Kent PDF in terms of spherical coordinates as used in equatorial coordinates.

$$
    \vec{x} = \begin{pmatrix}
                \cos\theta\cos{\varphi} \\ \cos\theta\sin{\varphi} \\ \sin\theta
              \end{pmatrix}
$$

In the integration we must not forget to multiply with the functional determinant (surface element) $\mathrm{d}A=-r^2\cos\varphi\mathrm{d}\theta\mathrm{d}\varphi$ for the convention used here.

Without loss of generality we can assume that the mean vector (=src position) is pointing along the z-axis $(0,0,1)$.
This way we have easy integration boundaries, the radius of the circle is then directly given by the polar angle $\theta$:

$$
    \begin{pmatrix} 0 \\ 0 \\ 1 \end{pmatrix} \cdot
      \begin{pmatrix} \cos\theta\cos{\varphi} \\ \cos\theta\sin{\varphi} \\ \sin\theta \end{pmatrix}
    = \sin\theta
$$

so our full integral assembles to:

\begin{align}
    \mathrm{Prob\ Content\ within\ }\sigma = \alpha &= \int_{\theta=\pi/2}^{\pi/2 - \sigma}\int_{\varphi=0}^{2\pi}
        \frac{\kappa}{4\pi\sinh{\kappa}}\cdot\exp(\kappa\sin\theta)\cdot(-\cos\theta)
        \mathrm{d}\theta\mathrm{d}\varphi \\
      &= 2\pi\cdot\frac{\kappa}{4\pi\sinh{\kappa}}\cdot \int_{\theta=\pi/2}^{\pi/2 - \sigma}
        -\cos\theta\exp(\kappa\sin\theta)\mathrm{d}\theta\mathrm{d}\varphi \\
      &= \frac{\kappa}{2\sinh{\kappa}}\cdot \int_{\theta=\pi/2 - \sigma}^{\pi/2}
        \cos\theta\exp(\kappa\sin\theta)\mathrm{d}\theta\mathrm{d}\varphi
\end{align}

We can solve this analytially:

\begin{align}
    \alpha &= \frac{1}{2\sinh{\kappa}}\cdot \int_{\theta=\pi/2 - \sigma}^{\pi/2}
               \kappa\cos\theta\exp(\kappa\sin\theta)\mathrm{d}\theta
            = \frac{1}{2\sinh{\kappa}}\cdot \left[\exp(\kappa\sin\theta)\right]_{\theta=\pi/2 - \sigma}^{\pi/2} \\
            &= \frac{1}{2\sinh{\kappa}}\cdot \left[\exp(\kappa) - \exp(\kappa\cos(\sigma))\right]
\end{align}

This automatically reduces to $\sinh\kappa / \sinh\kappa = 1$ for $\sigma=180°$ and to zero for $\sigma=0°$ as expected.

The special case $\kappa=0$ resembles a uniform distribtuion on the sky and is given by:

\begin{align}
    \alpha &= \int_{\theta=\pi/2 - \sigma}^{\pi/2}\int_{\varphi=0}^{2\pi}
               \frac{\cos\theta}{4\pi} \mathrm{d}\theta\mathrm{d}\varphi
           =\frac{1}{2}\int_{\theta=\pi/2 - \sigma}^{\pi/2}\cos\theta\mathrm{d}\theta \\
           &= \frac{\sin(\pi/2) - \sin(\pi/2-\sigma)}{2}
           = \frac{1 - \cos\sigma}{2}
\end{align}

Below we run some tests up to which events sigma we can go with the Kent distribution.
Also we see, if it is justified to identify $\kappa = 1/\sigma**2$.

In [ ]:
# Quickly see that the pdf itself is normalized.
# Note: Integral is in cos(theta), so no area element needed.
# For very large kappas this breaks because the stepsize is too coarse
def S(cosDist, kappa):
    return (kappa / (2. * np.pi * (1. - np.exp(-2. * kappa))) *
                 np.exp(kappa * (cosDist - 1. )))

sig = np.deg2rad(2)
kappa = 1 / sig**2
scint.quad(S, a=3*sig, b=1, args=(kappa))[0] * 2 * np.pi

In [ ]:
# First let's get the 1 sigma gaussian probability content to compare against
one_sig = scs.norm.cdf(1) - scs.norm.cdf(-1)

def alpha_in_kent_stable(kappa, sigma):
    """
    moxe is a numeric god :+1:
    """
    sigma = np.atleast_1d(sigma)
    if np.any(sigma > np.pi):
        raise ValueError("Angular error greater than pi.")
    if kappa == 0:
        return 0.5 * (1. - np.cos(sigma))

    a = 1. / np.tanh(kappa) + 1.
    b = 0.5 * (1. - np.exp(kappa * (np.cos(sigma) - 1.)))
    return a * b

def alpha_in_kent(kappa, sigma):
    sigma = np.atleast_1d(sigma)
    if np.any(sigma > np.pi):
        raise ValueError("Angular error greater than pi.")
    if kappa == 0:
        return 0.5 * (1. - np.cos(sigma))
    
    return 0.5 * (np.exp(kappa) - np.exp(kappa * np.cos(sigma))) / np.sinh(kappa)

def kappa_from_sigma(sigma, alpha=0.39):
    # Wrapper because we need the point alpha(k) - alpha = 0
    def fun(kappa):
        return alpha_in_kent_stable(kappa, sigma) - alpha
    
    x0 = float(1. / sigma**2)
    return sco.newton(fun, x0=x0)

In [ ]:
# Up to ~40° kappa = 1/sigma**2 is OK to use. Then the norm kicks in
sigmas = np.linspace(0.1, 150, 1000)
alpha = scs.chi2.cdf(1**2, df=2)

kappas = np.zeros_like(sigmas)
for i, sig in enumerate(sigmas):
    kappas[i] = kappa_from_sigma(np.deg2rad(sig), alpha=alpha)
    
# Simple subtitution as in mrichmans thesis
subst = 1. / np.deg2rad(sigmas)**2

plt.plot(sigmas, kappas, label=r"Exact")
plt.plot(sigmas, subst, label=r"$\kappa = 1/\sigma^2$")

plt.xlabel("sigma in °")
plt.ylabel(r"$\kappa$")
plt.yscale("log")
plt.legend()

# plt.savefig("data/figs/kent_kappa.png", dpi=200)

In [ ]:
# Different view of the plot above, now in terms of different kappas
sigmas = np.linspace(0, 20, 360 + 1)
sigmas_rad = np.deg2rad(sigmas)
kappas = np.arange(10, 200, 20)

alpha = scs.chi2.cdf(1**2, df=2)  # ca. 39%, 2D gaussian
plt.axhline(1, 0, 1, color=dg, ls="--")
plt.axhline(alpha, 0, 1, color=dg, ls="--")
for k in kappas:
    a = alpha_in_kent(k, sigmas_rad)
    plt.plot(sigmas, a, label=r"$\kappa = {:.1f}$".format(k))

# Note: if evt sigma is larger than ~80°, we can't describe the correct
#       probability content with a Kent distribution. An artifact from the
#       LLH beeing not gaussian enough.
plt.xlabel("Evt sigma in °")
plt.ylabel("Prob. in sigma")
plt.xlim(0, 20)
plt.ylim(0, 1.05)
plt.legend()
plt.title("CDF of symmetric Kent distribution")

# plt.savefig("data/figs/kent_cdf.png", dpi=200)

In [ ]:
# Now directly compare Kent to 2D gaussian (provided by mrichman :+1:)
# For sigmas near the turnover ~75°, we can see, that the Kent line crosses
# the gaussian. It smoothes out more to get exactly the 39% at the sigma level
mth = 90
theta = np.linspace (0, np.deg2rad(mth), 1000)
sig_deg = [3, 5, 10, 20, 30, 75]

cmap = plt.cm.get_cmap("plasma")

for sd in sig_deg:
    sr = np.radians(sd)
    # kappa = 1 / sr**2
    kappa = kappa_from_sigma(sigma=sr, alpha=scs.chi2.cdf(1**2, df=2))
    G = np.exp(-theta**2 / (2 * sr**2)) / (2. * np.pi * sr**2)
    K = kappa / (4. * np.pi * np.sinh(kappa)) * np.exp(kappa * np.cos(theta))
    color = cmap((sd - min(sig_deg)) / max(sig_deg))
    plt.plot(np.rad2deg(theta), K, color=color,
             label=r'$\sigma={}^\circ$'.format (sd))
    plt.plot(np.rad2deg(theta), G, ls='--', color=color)

plt.yscale("log")
plt.xlabel (r'$\Delta\Psi~[^\circ]$')
plt.ylabel (r'PDF')
plt.xlim (0, mth)
plt.ylim (1e-3, 1e3)
plt.legend (loc='best', ncol=2, prop={"size": 'small'}, title="Kent")
plt.tight_layout()

# plt.savefig("data/figs/kent_vs_gaus.png", dpi=200)

# Tests for the KDE integration

## Marginalize KDE by integration

Instead of sampling and reducing to 2D histograms, we can try to truly integrate one dimension of the KDE to be able to plot also the tails of the distribution, where events usually end up only in large samples.

In [ ]:
# Assign model from CV, which has already evaluated adaptive kernels
kde_inj = json2kde("data/awKDE_CV/CV10_glob_bw_alpha_EXP_IC86I_" +
                   "CUT_sig.ll.20_PARS_diag_True_pass2.json")

# Generate some BG samples to compare to the original data hist
nsamples_kde = int(1e7)
bg_samples = kde_inj.sample(nsamples_kde)

# Plot in degrees and in sinDec
bg_samples = np.vstack((bg_samples[:, 0],
                        np.sin(bg_samples[:, 1]),
                        np.rad2deg(bg_samples[:, 2]))).T

In [ ]:
# NOTE: The scaled version are super old and used a fixed width sklearn KDE
#       with mahual scaling of the input space. These are the scale factors:
fac_logE = 1.5
fac_dec = 2.5
fac_sigma = 2.

In [ ]:
# KDE CV is running on cluster and pickles the GridSearchCV
fname = "./data/kde_cv/KDE_model_selector_20_exp_IC86_I_followup_2nd_pass.pickle"
with open(fname, "rb") as f:
    model_selector = pickle.load(f)

kde = model_selector.best_estimator_
bw = model_selector.best_params_["bandwidth"]
print("Best bandwidth : {:.3f}".format(bw))

# We maybe just want to stick with the slightly overfitting kernel to
# be as close as possible to data
OVERFIT = True
if OVERFIT:
    bw = 0.075
    kde = skn.KernelDensity(bandwidth=bw, kernel="gaussian", rtol=1e-8)
print("Used bandwidth : {:.3f}".format(bw))

# KDE sample must be cut in sigma before fitting, similar to range in hist
_exp = exp[exp["sigma"] <= np.deg2rad(5)]

fac_logE = 1.5
fac_dec = 2.5
fac_sigma = 2.

_logE = fac_logE * _exp["logE"]
_sigma = fac_sigma * np.rad2deg(_exp["sigma"])
_dec = fac_dec * _exp["dec"]

kde_sample = np.vstack((_logE, _dec, _sigma)).T

# Fit KDE best model to sample
kde.fit(kde_sample)

# Make some samples
nsamples_kde = int(1e7)
bg_samples = kde.sample(n_samples=nsamples_kde)

### 1D case

Integrate out 2 axis with a double integral to show a 1D margin distribution.
This take super long, 5 Minutes per point.
But it can be parallelized pretty simple if needed.
Code to create the values is on phobos.

**Resumee:** Sampling many values and simply bin in 1D is much better, needs less time and is probably more accurate.

In [ ]:
# Compare to data hist
_ = plt.hist(exp["logE"], bins=100, normed=True, label="data")

# Compare 'true' integration with
h, b = np.histogram(bg_samples[:, 0], bins=200, range=[2, 6], normed=True)
_ = plt.plot(get_binmids(b), h, label="sample")

bins_and_vals = np.load("data/2d_integrate_kde_SCALED/bins_and_vals.npy")
x = bins_and_vals[0] / fac_logE
vals = bins_and_vals[1]
_ = plt.plot(x, vals, label="integrated")

plt.legend()

### 2D PDF

Integrate out only one axis (here sigma) to show the 2D marginalized PDF.
Executed on phobos, takes a little while on 50x50 grid.

In [ ]:
# Get precalculated integral data
bins = list(np.load("data/1d_integrate_kde_SCALED/logE_sinDec_bins_50x50.npy"))
vals = np.load("data/1d_integrate_kde_SCALED/logE_sinDec_int_50x50.npy")

fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Compare to data hist
_ = ax[0].hist2d(exp["logE"], np.sin(exp["dec"]), bins=bins, normed=True)

# Compare 'true' integration
mids = get_binmids(bins)
xx, yy  = map(np.ravel, np.meshgrid(mids[0], mids[1]))
_ = ax[1].hist2d(xx, yy, bins=bins, weights=vals)

fig.tight_layout()

## Justify the sigma cut

Only few higher energy events from the sothern sky are excluded (see cut=10).
But really bad reconstructed events tend to have higher energies (see cut=90).
Still it should be OK to remove those > 10 because they have not so much spatial information.

In [ ]:
# Show the leftover event s after a sigma cut
sig_cut = 10
m = exp["sigma"] > np.deg2rad(sig_cut)

_ = plt.hist2d(exp["logE"][m], np.rad2deg(exp["dec"][m]),
               bins=30, cmap="inferno")
plt.colorbar()
plt.title("Total Evts w sigma > {:d}°: {:d} ({:.3f}%)".format(
        sig_cut, np.sum(m), np.sum(m) / len(exp) * 100))
plt.xlabel("logE")
plt.ylabel("dec in °")
plt.show()

# Show the skewed sigma distribution with the cut applied and mean vs median

## Test the marginalize_hist method

It should be equivalent to use one of the following methods to create a 1D histogram from the original 3D data pdf in logE, dec and sigma:

1. Simply use the original 1D data in any variable, e.g. simply histogram logE
2. Create the complete 3D histogram and marginalize by summing over remaining dimensions.

When using unnormalized hists, 2. is simply summing up all other counts.

When using normalized hists, we need to sum with respect to the binwidths in the current dimension to keep the normalization intact.
This is only useful, when only the histogram is available and not the original sample.

We want to compare if both methods are equivalent
As we can see, all ratios are one, so methods are equal.

In [ ]:
def make_hist_ratio(h1, h2):
    """Return the ratio h1 / h2. Return 0 where h2 is 0."""
    m = (h2 > 0)
    ratio = np.zeros_like(h1)
    ratio[m] = h1[m] / h2[m]
    return ratio

### Unnormalized
First the unnormalized version. Simply sum over the other axes of the 3D hist.

In [ ]:
# Plot each variable in a single plot and the ratios seperately
fig, [[axtl, axtr], [axbl, axbr]] = plt.subplots(2, 2, figsize=(10, 8))

# We also make a cut < 10° in sigma, because there are some outliers
m = exp["sigma"] <= np.deg2rad(10)
sigma = np.rad2deg(exp["sigma"][m])
logE = exp["logE"][m]
dec = np.sin(exp["dec"][m])

logE_nbins = 50
dec_nbins = 40
sigma_nbins = 30

# Make the 3D hist
sample = np.vstack((logE, dec, sigma)).T
nbins = [logE_nbins, dec_nbins, sigma_nbins]
h, b = np.histogramdd(sample, bins=nbins,)

# Get binmids for plotting
m = get_binmids(b)

# Common hist settings
h1 = {"lw": 2, "color": "k", "histtype": "step"}
h2 = {"lw": 2, "color": "r", "histtype": "step", "alpha": 0.5}

# logE
logE_h, logE_b, _ = axtl.hist(logE, bins=logE_nbins, **h1)
logE_hm = np.sum(h, axis=(1, 2))
_ = axtl.hist(m[0], bins=b[0], weights=logE_hm, **h2)
# Ratio plot below
axtl_sec = split_axis(axtl, "bottom", "20%", cbar=False)
axtl_sec.hist(m[0], b[0], weights=make_hist_ratio(logE_h, logE_hm), **h2)
axtl_sec.axhline(1, 0, 1, color="k")
axtl_sec.set_ylim(0, 2)

# dec
dec_h, dec_b, _ = axbl.hist(dec, bins=dec_nbins, **h1)
dec_hm = np.sum(h, axis=(0, 2))
_ = axbl.hist(m[1], bins=b[1], weights=dec_hm, **h2)

axbl_sec = split_axis(axbl, "bottom", "20%", cbar=None)
axbl_sec.hist(m[1], b[1], weights=make_hist_ratio(dec_h, dec_hm), **h2)
axbl_sec.axhline(1, 0, 1, color="k")
axbl_sec.set_ylim(0, 2)

# sigma
sigma_h, sigma_b, _ = axtr.hist(sigma, bins=sigma_nbins, **h1)
sigma_hm = np.sum(h, axis=(0, 1))
_ = axtr.hist(m[2], bins=b[2], weights=sigma_hm, **h2)

axtr_sec = split_axis(axtr, "bottom", "20%", cbar=None)
axtr_sec.hist(m[2], b[2], weights=make_hist_ratio(sigma_h, sigma_hm), **h2)
axtr_sec.axhline(1, 0, 1, color="k")
axtr_sec.set_ylim(0, 2)

axbr.set_visible(False)

fig.suptitle("Black: 1D, Red: Margin", fontsize=15);

### Normalized
Sum over the other axes of the 3D hist and multiply by bin widths.

In [ ]:
# Plot each variable in a single plot and the ratios seperately
fig, [[axtl, axtr], [axbl, axbr]] = plt.subplots(2, 2, figsize=(10, 8))

# Now make it normed
h, b = np.histogramdd(sample, bins=nbins, normed=True)

# Get binmids for plotting
m = get_binmids(b)

# logE
logE_h, logE_b, _ = axtl.hist(logE, bins=logE_nbins, normed=True, **h1)
logE_hm = hist_marginalize(h=h, bins=b, axes=(1, 2))[0]
_ = axtl.hist(m[0], bins=b[0], weights=logE_hm, **h2)
# Ratio plot below
axtl_sec = split_axis(axtl, "bottom", "20%", cbar=False)
axtl_sec.hist(m[0], b[0], weights=make_hist_ratio(logE_h, logE_hm), **h2)
axtl_sec.axhline(1, 0, 1, color="k")
axtl_sec.set_ylim(0, 2)

# dec
dec_h, dec_b, _ = axbl.hist(dec, bins=dec_nbins, normed=True, **h1)
dec_hm = hist_marginalize(h=h, bins=b, axes=(0, 2))[0]
_ = axbl.hist(m[1], bins=b[1], weights=dec_hm, **h2)

axbl_sec = split_axis(axbl, "bottom", "20%", cbar=None)
axbl_sec.hist(m[1], b[1], weights=make_hist_ratio(dec_h, dec_hm), **h2)
axbl_sec.axhline(1, 0, 1, color="k")
axbl_sec.set_ylim(0, 2)

# sigma
sigma_h, sigma_b, _ = axtr.hist(sigma, bins=sigma_nbins, normed=True, **h1)
sigma_hm = hist_marginalize(h=h, bins=b, axes=(0, 1))[0]
_ = axtr.hist(m[2], bins=b[2], weights=sigma_hm, **h2)

axtr_sec = split_axis(axtr, "bottom", "20%", cbar=None)
axtr_sec.hist(m[2], b[2], weights=make_hist_ratio(sigma_h, sigma_hm), **h2)
axtr_sec.axhline(1, 0, 1, color="k")
axtr_sec.set_ylim(0, 2)

axbr.set_visible(False)

fig.suptitle("Black: 1D, Red: Margin", fontsize=15);

# Mask sinDec logE ratio PDF spline

Assign each high E events for which no data but MC is present the maximum ratio from signal over background and all lowE the lowest ratio.

In skylab all events which no data but MC events get the highest value, which is strange at the lowE regime.

In [ ]:
# Prepare the MC data, signal weighted to astro unbroken power law
gamma = 2.
# No flux norm, because we normalize anyway
mc_w = mc["ow"] * mc["trueE"]**(-gamma)

# Make the MC hist. Use this binning for the data too
mc_sindec = np.sin(mc["dec"])
mc_logE = mc["logE"]
bins = [50, 50]
rnge = [[-1, 1], [1, 10]]
mc_h, bx, by = np.histogram2d(mc_sindec, mc_logE, bins=bins, range=rnge,
                              weights=mc_w, normed=True)

# Make the data hist
b = [bx, by]
bg_logE = exp["logE"]
bg_sindec = np.sin(exp["dec"])
bg_h, _, _ = np.histogram2d(bg_sindec, bg_logE, bins=b,
                                range=rnge, normed=True)


# 4 cases:
#   - Data & MC: Calculate the ratio
#   - (>logEthresh) & (no data or no MC): Assign highest normal ratio
#   - (<logEthresh) & (no data or no MC): Assign lowest normal ratio
#   - No data and no MC): Assign any value (eg 1), these are never accessed
# Get logE value per bin in entrie histogram
m = get_binmids(b)
logEs =  np.repeat(m[1], repeats=bins[1]).reshape(bins).T
logEthresh = 3.5

m1 = (bg_h > 0) & (mc_h > 0)
m2 = (logEs > logEthresh) & ((bg_h <= 0) | (mc_h <= 0))
m3 = (logEs <= logEthresh) & ((bg_h <= 0) | (mc_h <= 0))
m4 = (bg_h <= 0) & (mc_h <= 0)

SoB = np.ones_like(bg_h)
SoB[m1] = mc_h[m1] / bg_h[m1]
SoB[m2] = np.amax(SoB)
SoB[m3] = np.amin(SoB)
SoB[m4] = 1.

# Make a 4 color mask map
mask_map = np.ones_like(bg_h)
mask_map[m1] = 1.
mask_map[m2] = 2.
mask_map[m3] = 0.
mask_map[m4] = -1.

# Plot it
fig, ((axtl, axtr), (axbl, axbr)) = plt.subplots(2, 2, figsize=(12, 10))
xx, yy = map(np.ravel, np.meshgrid(*m))

_ = axtl.hist2d(xx, yy, bins=b, weights=bg_h.T.flatten(), cmap="viridis",
               norm=LogNorm())
_ = axtr.hist2d(xx, yy, bins=b, weights=mc_h.T.flatten(), cmap="viridis",
               norm=LogNorm())

# Plot mask
cbins = np.linspace(-1, 2, 5)
cticks = 0.5 * (cbins[:-1] + cbins[1:])
cmap = matplotlib.cm.get_cmap("inferno", len(cticks))
_, _, _, img = axbr.hist2d(xx, yy, bins=b, weights=mask_map.T.flatten(),
                          cmap=cmap)
cax = split_axis(axbr, cbar=True)
cbar = plt.colorbar(cax=cax, mappable=img, ticks=cticks)
cbar.ax.set_yticklabels(["No BG AND no MC", "lowE, no D OR MC", "BG AND MC",
                         "higE, no D OR MC"],
                        rotation=60, va="center")

# Plot ratio
cn = max(np.amin(SoB), np.amax(SoB))
_, _, _, img = axbl.hist2d(xx, yy, bins=b, weights=SoB.T.flatten(),
                          cmap="coolwarm", vmin=1./cn, vmax=cn, norm=LogNorm())
cax = split_axis(axbl, cbar=True)
cbar = plt.colorbar(cax=cax, mappable=img)

fig.tight_layout()
# plt.savefig("./data/figs/energy_ratio_mask.png", dpi=200)
plt.show()

# Compare sigma to x*exp(-x)

In [ ]:
# Data
sig_deg = np.rad2deg(exp["sigma"][exp["sigma"] < np.deg2rad(10)])
_, b, _ = plt.hist(sig_deg, bins=50, normed=True)

# PDF
x = np.linspace(0, 10, 500)
a = 3
y = a**2 * x * np.exp(-a * x)
_ = plt.plot(x, y, lw=2)

# Sampled from PDF
# Combo from Pythia 8 and Trial&Error. How to derive from LambertW function?
#   http://home.thep.lu.se/~torbjorn/doxygen/Basics_8h_source.html
u1, u2 = np.random.uniform(size=(2, 10000))
sam = -np.log(u1 * u2) / a
plt.hist(sam, bins=b, normed=True, histtype="step", lw=2)

plt.xlim(0, 5)

# Old Scaled KDE

To save the old non-adaptive and kind-of guessed KDE for comparison, we put the former code in here.

## 3D histogram of BG data
First we make a 3D histogram to better compare to mrichmann and to get an overview over the distribution.

In [ ]:
# HANDTUNED scale parameter to "fit" KDE expectation to data...
# TODO: Use Adaptive kernel width and asymmetric gaus kernels
#       For sigma it might make sense to a take a restricted kernel [0, inf]
fac_logE = 1.5
fac_dec = 2.5
fac_sigma = 2.

logE = fac_logE * exp["logE"]
sigma = fac_sigma * np.rad2deg(exp["sigma"])
# np.cos(np.pi / 2. + exp["dec"]); dec is for {sin(dec), dec, cos(zen)}
dec = fac_dec * exp["dec"]

# Binning is rather arbitrary because we don't calc stuff with the hist
bins = [50, 50, 50]
# Range for sigma is picked by looking at the 1D distribution and cutting of
# the tail. This will be covered by the KDE tail anyway. Rest is default
r = [[np.amin(logE), np.amax(logE)],
     [np.amin(dec), np.amax(dec)],
     [0., fac_sigma * 5.]]

sample = np.vstack((logE, dec, sigma)).T
h, bins = np.histogramdd(sample=sample, bins=bins, range=r, normed=False)

# Make bin mids for later use
mids = []
for b in bins:
    mids.append(0.5 * (b[:-1] + b[1:]))

# Make a nice corner plot
fig, ax = hlp.corner_hist(h, bins=bins,
                          label=["scaled logE", "scaled dec", "scaled sigma deg"],
                          hist2D_args={"cmap": "Greys"},
                          hist_args={"color":"#353132"})

# plt.savefig("./data/figs/bg_corner_scaled.png", dpi=200)

## Kernel Density Estimation

We use scikit learn's cross validation with a gaussian kernel to get the most robust bandwidth.
Then we integrate with the same binning as above and compare to the 3D histogram.

This section relies heavily on [Jake van der Plas examples for KDE](https://jakevdp.github.io/blog/2013/12/01/kernel-density-estimation/).
More info on how KDE cross validation works can be found in [Modern Nonparametric Methods](http://www2.stat.duke.edu/~wjang/teaching/S05-293/lecture/ch6.pdf).

In [ ]:
with open("data/kde_cv_SCALED/KDE_model_selector_20_exp_IC86_I_" +
          "followup_2nd_pass.json", "r") as f:
    kde_params = json.load(f)
    
kde = skn.KernelDensity(**kde_params)

# KDE sample must be cut in sigma before fitting, similar to range in hist
_exp = exp[exp["sigma"] <= np.deg2rad(5)]

_logE = fac_logE * _exp["logE"]
_sigma = fac_sigma * np.rad2deg(_exp["sigma"])
_dec = fac_dec * _exp["dec"]

kde_sample = np.vstack((_logE, _dec, _sigma)).T

# Fit KDE best model to sample
kde.fit(kde_sample)

# Generate some BG samples to compare to the original data hist.
# Use more statistics, histograms get normalized and we want the best estimate
# for the pdf
nsamples_kde = int(1e7)
bg_samples = kde.sample(nsamples_kde)

# Make histogram with same binning as original data
bg_h, bg_bins = np.histogramdd(sample=bg_samples, bins=bins, range=r, normed=True)

fig, ax = hlp.corner_hist(bg_h, bins=bg_bins,
                         label=["scaled logE", "scaled sin(dec)",
                                "scaled sigma deg"],
                         hist2D_args={"cmap": "Greys"},
                         hist_args={"color":"#353132"})

# plt.savefig("./data/figs/kde_corner_scaled.png", dpi=200)

## Compare KDE to original data

Make a ratio histogram of the KDE sample and the original data sample.

### 2D marginalization

In [ ]:
# Create 2D hists, by leaving out one parameter
xlabel = ["scaled " + s for s in ["logE", "logE", "dec"]]
ylabel = ["scaled " + s for s in ["dec", "sigma in °", "sigma in °"]]

for i, axes in enumerate([[0, 1], [0, 2], [1, 2]]):
    _b = np.array(bins)
    h_exp, b_exp = np.histogramdd(sample[:, axes],
                                  bins=_b[axes], normed=True)
    h_kde, b_kde = np.histogramdd(bg_samples[:, axes],
                                  bins=_b[axes], normed=True)
    
    # KDE is expectation, but sampled with much more events.
    # Weights would simply scale the total number of KDE events to match the
    # number of original events. That would be the mean for the poisson
    # distribution in each bin. So to get OK KDE expectation sqrt(n) errors
    # in each bin, we divide not by the number of drawn KDE but by the number
    # of original events.   
    # Again shapes of meshgrid and hist are transposed
    diffXX, _ = np.meshgrid(np.diff(_b[0]), np.diff(_b[1]))
    norm_kde = len(exp) * diffXX.T
    sigma_kde = np.sqrt(h_kde / norm_kde)

    # Make 3 different diff/ratio hists to estimate KDE quality in
    # 1D marginalization.
    m = (h_exp > 0.)
    ratio_h = np.zeros_like(h_exp)
    ratio_h[m] = h_kde[m] / h_exp[m]

    diff_h = h_kde - h_exp

    m = (sigma_kde > 0.)
    sigma_ratio_h = np.zeros_like(h_exp)
    sigma_ratio_h[m] = (h_exp[m] - h_kde[m]) / sigma_kde[m]

    # Bin mids and hist grid
    _b = b_exp
    m = get_binmids(_b)
    xx, yy = map(np.ravel, np.meshgrid(m[0], m[1]))
    
    
    # Big plot on the left and three right
    fig = plt.figure(figsize=(10, 6))
    gs = gridspec.GridSpec(3, 3)
    axl = fig.add_subplot(gs[:, :2])
    axrt = fig.add_subplot(gs[0, 2])
    axrc = fig.add_subplot(gs[1, 2])
    axrb = fig.add_subplot(gs[2, 2])
    
    # Steal space for colorbars
    caxl = split_axis(axl, "right")
    caxrt = split_axis(axrt, "left")
    caxrc = split_axis(axrc, "left")
    caxrb = split_axis(axrb, "left")

    # Unset top and center xticklabels as they are shared with the bottom plot
    axrt.set_xticklabels([])
    axrc.set_xticklabels([])
        
    # Left: Difference over KDE sigma
    # cbar_extr = max(np.amax(sigma_ratio_h),  # Center colormap to min/max
    #                         abs(np.amin(sigma_ratio_h)))
    _, _, _, imgl = axl.hist2d(xx, yy, bins=_b, weights=sigma_ratio_h.T.ravel(),
                               cmap="seismic", vmax=5, vmin=-5)
    cbarl = plt.colorbar(cax=caxl, mappable=imgl)
    axl.set_xlabel(xlabel[i])
    axl.set_ylabel(ylabel[i])
    axl.set_title("(exp - kde) / sigma_kde")
    
    # Right top: Ratio
    _, _, _, imgrt = axrt.hist2d(xx, yy, bins=_b, weights=ratio_h.T.ravel(),
                                 cmap="seismic", vmax=2, vmin=0);
    cbarrt = plt.colorbar(cax=caxrt, mappable=imgrt)
    axrt.set_title("kde / exp")

    # Right center: Data hist
    _, _, _, imgrc = axrc.hist2d(xx, yy, bins=_b, weights=h_exp.T.ravel(),
                                 cmap="Greys", norm=LogNorm());
    cbarrc = plt.colorbar(cax=caxrc, mappable=imgrc)
    axrc.set_title("exp logscale")

    # Right bottom: KDE hist, same colorbar scale as on data
    _, _, _, imgrb = axrb.hist2d(xx, yy, bins=_b, weights=h_kde.T.ravel(),
                                 cmap="Greys", norm=LogNorm());
    # Set with same colormap as on data
    imgrb.set_clim(cbarrc.get_clim())
    cbarrb = plt.colorbar(cax=caxrb, mappable=imgrb)
    axrb.set_title("kde logscale")
    
    # Set tick and label positions
    for ax in [caxrt, caxrc, caxrb]:
        ax.yaxis.set_label_position("right")
        ax.yaxis.tick_left()
    
    fig.tight_layout()
#     plt.savefig("./data/figs/kde_data_2d_{}_{}.png".format(
#                     xlabel[i], ylabel[i]), dpi=200)
    plt.show()

### 1D marginalization

In [ ]:
# Pseudo smooth marginalization is done by sampling many point from KDE an
# using a finely binned 1D histogram, so it looks smooth
xlabel = ["scaled " + s for s in ["logE", "dec", "sigma °"]]

for i, axes in enumerate([0, 1, 2]):
    _b = np.array(bins)
    h_exp, b_exp = np.histogram(sample[:, axes],
                                bins=_b[axes], normed=True)
    h_kde, b_kde = np.histogram(bg_samples[:, axes],
                                bins=_b[axes], normed=True)
    
#     h_exp, b_exp = hist_marginalize(h, bins, axes=axes)
#     h_kde, b_kde = hist_marginalize(bg_h, bg_bins, axes=axes)
      
    # KDE errorbars as in 2D case
    norm_kde = len(exp) * np.diff(b_kde)
    sigma_kde = np.sqrt(h_kde / norm_kde)

    # Make 3 different diff/ratio hists to estimate KDE quality in
    # 1D marginalization.
    m = (h_exp > 0.)
    ratio_h = np.zeros_like(h_exp)
    ratio_h[m] = h_kde[m] / h_exp[m]

    diff_h = h_kde - h_exp

    m = (sigma_kde > 0.)
    sigma_ratio_h = np.zeros_like(h_exp)
    sigma_ratio_h[m] = (h_exp[m] - h_kde[m]) / sigma_kde[m]

    # Bin mids
    _b = b_exp
    m = get_binmids([_b])[0]
    
    # Plot both and the ration normed. Big plot on the left and three right
    fig = plt.figure(figsize=(10, 6))
    gs = gridspec.GridSpec(3, 3)
    axl = fig.add_subplot(gs[:, :2])
    axrt = fig.add_subplot(gs[0, 2])
    axrc = fig.add_subplot(gs[1, 2])
    axrb = fig.add_subplot(gs[2, 2])

    axrt.set_xticklabels([])
    axrc.set_xticklabels([])

    # Set ticks and labels right
    for ax in [axrt, axrc, axrb]:
        ax.yaxis.set_label_position("right")
        ax.yaxis.tick_right()

    # Limits
    for ax in [axl, axrt, axrc, axrb]:
        ax.set_xlim(_b[0], _b[-1])
        
    # Main plot:
    # Plot more dense to mimic a smooth curve
    __h, __b = np.histogram(bg_samples[:, i], bins=500,
                            range=[_b[0], _b[-1]], density=True)
    __m = get_binmids([__b])[0]
    axl.plot(__m, __h, lw=3, alpha=0.5)
    
    _ = axl.hist(m, bins=_b, weights=h_exp, label="exp", histtype="step",
                 lw=2, color="k")
    _ = axl.errorbar(m, h_kde, yerr=sigma_kde, fmt=",", color="r")
    _ = axl.hist(m, bins=_b, weights=h_kde, label="kde", histtype="step",
                 lw=2, color="r")    
    
    axl.set_xlabel(xlabel[i])
    axl.legend(loc="upper right")

    # Top right: Difference
    _ = axrt.axhline(0, 0, 1, color="k", ls="-")
    _ = axrt.hlines([-.02, -.01, .01, .02], _b[0], _b[-1],
                    colors='#353132', linestyles='dashed')
    _ = axrt.hist(m, bins=_b, weights=diff_h, histtype="step", lw=2, color="r")
    axrt.set_ylim(-.05, +.05)
    axrt.set_ylabel("kde - exp")

    # Center right: Ratio
    _ = axrc.axhline(1, 0, 1, color="k", ls="-")
    _ = axrc.hlines([0.8, 0.9, 1.1, 1.2], _b[0], _b[-1],
                    colors='#353132', linestyles='dashed')
    _ = axrc.hist(m, bins=_b, weights=ratio_h, histtype="step", lw=2, color="r")
    axrc.set_ylim(.5, 1.5)
    axrc.set_ylabel("kde / exp")

    # Bottom right: Ratio of diff to sigma of expectation
    _ = axrb.axhline(0, 0, 1, color="k", ls="-")
    _ = axrb.hlines([-2, -1, 1, 2], _b[0], _b[-1],
                    colors='#353132', linestyles='dashed')
    _ = axrb.hist(m, bins=_b, weights=sigma_ratio_h, histtype="step", lw=2, color="r")
    axrb.set_ylim(-3, +3)
    axrb.set_ylabel("(exp-kde)/sigma_kde")
    
#     plt.savefig("./data/figs/kde_data_1d_{}.png".format(
#             xlabel[i]), dpi=200)
    plt.show()

# Show scores of awKDE CV

In [ ]:
with open("data/awKDE_CV/CV10_glob_bw_alpha_EXP_IC86I_CUT_sig.ll.20_PARS" +
          "_diag_True_pass2_cv_res.json", "r") as f:
    cv_results = json.load(f)
    
glob_bws = cv_results["param_glob_bw"]
mean_score = cv_results["mean_test_score"]
std_score = cv_results["std_test_score"]

# Mark best
bf_idx = np.argmax(mean_score)
_ = plt.axvline(glob_bws[bf_idx], 0, 1, ls="--", color="C2", zorder=-1)
_ = plt.axhline(mean_score[bf_idx], 0, 1, ls="--", color="C2", zorder=-1)
# Each horizontal line is one alpha tested for that glob_bw
bf_idx = np.argmax(mean_score)
_ = plt.errorbar(glob_bws, mean_score, yerr=std_score,fmt="_")

plt.xlabel("global bandwidth")

# Old BG Rate Sampler

Tried to sample everything at once (trials, src, times).
Very clumsy because for each src and trial, a different number of evt is sampled, so we can't use arrays.

Also we can fit a single LLH at a time only, so no need to get all samples in advance.
Might be faster, but consumes also much more memory.

## Sample number of events in frame

In [ ]:
def filter_runs(run):
    """
    Filter runs as stated in jfeintzig's doc.
    """
    exclude_runs = [120028, 120029, 120030, 120087, 120156, 120157]
    if ((run["good_i3"] == True) & (run["good_it"] == True) &
        (run["run"] not in exclude_runs)):
        return True
    else:
        return False
    
goodrun_dict, _livetime = hlp.create_goodrun_dict(
    runlist="data/runlists/ic86-i-goodrunlist.json", filter_runs=filter_runs)

h = hlp._create_runtime_bins(exp["timeMJD"], goodrun_dict=goodrun_dict,
                             remove_zero_runs=True)

def f(x, pars):
    """
    Returns the rate at a given time in MJD.
    """
    a, b, c, d = pars
    return a * np.sin(b * (x - c)) + d

def lstsq(pars, *args):
    """
    Weighted leastsquares min sum((wi * (yi - fi))**2)
    """
    # data x,y-values and weights are fixed
    x, y, w = args
    _f = f(x, pars)
    return np.sum((w * (y - _f))**2)

# Seed values from consideration above.
# a0 = -0.0005
# b0 = 2. * np.pi / 365.  # We could restrict the period to one yr exact.
# c0 = np.amin(start_mjd)
# d0 = np.average(h, weights=yerr**2)

rate = h["rate"]
rate_std = h["rate_std"]
X = exp["timeMJD"]
binmids = 0.5 * (h["start_mjd"] + h["stop_mjd"])

a0 = 0.5 * (np.amax(rate) - np.amin(rate))
b0 = 2. * np.pi / 365.
c0 = np.amin(X)
d0 = np.average(rate, weights=rate_std**2)

x0 = [a0, b0, c0, d0]
# Bounds as explained above
bounds = [[None, None], [0.5 * b0, 1.5 * b0], [c0 - b0, c0 + b0, ], [0, 0.01]]
# x, y values, weights
args = (binmids, rate, 1. / rate_std)

res = sco.minimize(fun=lstsq, x0=x0, args=args, bounds=bounds)
bf_pars = res.x

print("Amplitude   : {: 13.5f} in Hz".format(res.x[0]))
print("Period (d)  : {: 13.5f} in days".format(2 * np.pi / res.x[1]))
print("Offset (MJD): {: 13.5f} in MJD".format(res.x[2]))
print("Avg. rate   : {: 13.5f} in Hz".format(res.x[3]))

# Define the rate function:
def rate_fun(t):
    """
    Returns the rate at a given time in MJD.
    
    Parameters
    ----------
    t : array-like
        Time in MJD.
        
    Returns
    -------
    rate : array-like
        The rate of background events in Hz.
    """
    return f(t, res.x)

In [ ]:
def _prep_times(t, trange):
    """
    Little wrapper to not DRY.
    """
    t = np.atleast_1d(t)
    trange = np.array(trange)
    nsrc = len(t)
    
    # Make shape (nsources, 1) for the times
    t = t.reshape(nsrc, 1)
    
    # If range is 1D (one for all) reshape it to (nsources, 2)
    if len(trange.shape) == 1:
        trange = np.repeat(trange.reshape(1, 2), repeats=nsrc, axis=0)
        
    # Prepare time window in MJD
    trange = t + trange / secinday
    
    return t, trange

def get_num_of_bg_events(t, trange, ntrials, pars):
    """
    Draw number of background events per trial from a poisson distribution
    with the mean of the fitted rate function.
    Then draw nevents times via rejection sampling for the time dpeendent rate
    function.
    
    Parameters
    ----------
    t : array-like
        Times of the occurance of each source event in MJD.
    trange : [float, float] or array_like, shape (len(t), 2)
        Time window(s) in seconds relativ to the given time(s) t.
        - If [float, float], the same window [lower, upper] is used for every
          source.
        - If array-like, lower [i, 0] and upper [i, 1] bounds of the time
          window per source.
    ntrials : int
        Number of background trials we need the number of how many events to
        inject for.
    pars : array-like
        Best fit parameters from the fit function used in its integral.
        
    Returns
    -------
    nevents : array-like, shape (len(t), ntrials)
        The number of events to inject for each trial for each source.
    """
    # Integrate rate function analytially in desired interval
    def rate_integral(trange, pars):
        """
        Match with factor [secinday] = 24 * 60 * 60 s / MJD = 86400/(Hz*MJD)
        in the last step.
            [a], [d] = Hz, [b], [c], [ti] = MJD
            [a / b] = Hz * MJD, [d * (t1 - t0)] = HZ * MJD
        """
        a, b, c, d = pars
        
        t0 = np.atleast_2d(trange[:, 0]).reshape(len(trange), 1)
        t1 = np.atleast_2d(trange[:, 1]).reshape(len(trange), 1)
        
        per = a / b * (np.cos(b * (t0 - c)) - np.cos(b * (t1 - c)))
        lin = d * (t1 - t0)

        return (per + lin) * secinday
    
    t, trange = _prep_times(t, trange)
    
    # Expectation is the integral in the time frame
    expect = rate_integral(trange, pars)
        
    # Sample from poisson
    nevts = np.random.poisson(lam=expect, size=(len(t), ntrials))
      
    return nevts

In [ ]:
start, stop = h["start_mjd"], h["stop_mjd"]

nsrc = 5
t = start[100:100 + nsrc]
# Make different time windows to verify that more events are sampled in more time
trange = np.array([[-10 * 5 * i, 20 * 5*i] for i in range(1, nsrc + 1)])
print("Time windows:\n", trange)

ntrials = 10
nevts = get_num_of_bg_events(t, trange, ntrials, res.x)
# print("All evts, per src and trial:\n", nevts)

# Total evts in all trials per src
print("Total sampled evts per src:\n", np.sum(nevts, axis=1)[:, np.newaxis])

# Compare to expectation
print("Expected per trial per src:\n", np.diff(trange, axis=1) * res.x[3])
print("Sampled per trial per src:\n", np.sum(nevts, axis=1)[:, np.newaxis] / ntrials)

## Now the sampling of random times in the time frame

First we want to see, that all BG injected events stay in the correct time frame and make a uniform distribution for small time frames.

Then we make the time window really big and the events should follow the rate function PDF.

In [ ]:
def get_times_in_frame(t, trange, nsamples):
    """
    Parameters
    ----------
    t : float
        Time of the occurance of the source event in MJD.
    trange : [float, float]
        Time window in seconds relativ to the given time t.
    nsamples : array-like, type int, shape (len(t))
        Number of events to inject per trial. Number of trials is given by
        the length of nsamples.
        
    Returns
    -------
    times : list, length len(t)
        List of samples times in MJD of background events per source.
        For each source i nsamples[i] times are drawn from the rate function.
    """
    _pdf = rate_fun
    
    t, trange = _prep_times(t, trange)
    
    sample = []
    nsamples = np.atleast_1d(nsamples)
    
    for i, ni in enumerate(nsamples):
        sam, _ = rejection_sampling(_pdf, bounds=trange, n=ni)
        sample.append(sam)
        
    return sample

In [ ]:
# First the small time frame
# Arbitrary start date from data
t0 = start[100]
t0_sec = t0 * secinday

# dt from t0 in seconds, clip at 4 sigma
dt = 200
nsig = 4.

# Make t values for plotting in MJD around t0
clip = np.clip(dt, 2, 30) * nsig
plt_rng = [-clip, dt + clip]
trange = plt_rng
ntrials = 10000

# Sample times
nevts = get_num_of_bg_events(t=t0, trange=trange, ntrials=ntrials,
                             pars=res.x)[0]
times = get_times_in_frame(t0, trange, nevts)

# Plot them in together with the PDFs
def time_bg_pdf(t, t0, a, b):
    # Normalize relative to t0 in seconds (first multiply avoids rounding?)
    _t = t * secinday - t0 * secinday
  
    pdf = np.zeros_like(_t, dtype=np.float)
    uni = (_t >= a) & (_t <= b)
    pdf[uni] = 1. / (b - a)
    return pdf

def time_sig_pdf(t, t0, dt, nsig=4):
    if dt < 0:
        raise ValueError("dt must not be negative.")

    # Normalize relative to t0 in seconds (first multiply avoids rounding?)
    _t = t * secinday - t0 * secinday
    
    # Constrain sig_t to [2, 30]s regardless of uniform time window
    sig_t = np.clip(dt, 2, 30)
    sig_t_clip = nsig * sig_t
    gaus_norm = (np.sqrt(2 * np.pi) * sig_t)
    
    # Split in def regions gaus rising, uniform, gaus falling and zero
    gr = (_t < 0) & (_t >= -sig_t_clip)
    gf = (_t > dt) & (_t <= dt + sig_t_clip)
    uni = (_t >= 0) & (_t <= dt)
    
    pdf = np.zeros_like(t, dtype=np.float)
    pdf[gr] = scs.norm.pdf(_t[gr], loc=0, scale=sig_t)
    pdf[gf] = scs.norm.pdf(_t[gf], loc=dt, scale=sig_t)
    # Connect smoothly with the gaussians
    pdf[uni] = 1. / gaus_norm
    
    # Normalize whole distribtuion
    dcdf = (scs.norm.cdf(dt + sig_t_clip, loc=dt, scale=sig_t) -
            scs.norm.cdf(-sig_t_clip, loc=0., scale=sig_t))
    norm = dcdf + dt / gaus_norm
    
    return pdf / norm


# Plot the pdfs
t = np.linspace(t0_sec + plt_rng[0], t0_sec + plt_rng[1], 200) / secinday
bg_pdf = time_bg_pdf(t, t0, -clip, dt + clip)
sig_pdf = time_sig_pdf(t, t0, dt, nsig)

# Plot in normalized time
_t = t * secinday - t0 * secinday
plt.plot(_t, bg_pdf, "C0-")
plt.plot(_t, sig_pdf, "C1-")
plt.axvline(dt, 0, 1, color="C7", ls="--")
plt.axvline(0, 0, 1, color="C1", ls="--")

# Plot injected events from all trials
T = np.array([])
for ti in times:
    T = np.append(T, ti)  
T = (T - t0) * secinday

_ = plt.hist(T, bins=50, normed=True, color=dg, alpha=.25)

plt.xlabel("Time relative to t0 in sec")
plt.ylim(0, None);
plt.tight_layout()

# plt.savefig("./data/figs/bg_events_time_sampled_narrow.png", dpi=200)

plt.show()

In [ ]:
# Now the really large time frame, over the whole time range
t0 = start[0]
t0_sec = t0 * secinday

# dt from t0 in seconds, clip at 4 sigma
dt = (stop[-1] - start[0]) * secinday
nsig = 4.

# Make t values for plotting in MJD around t0
clip = np.clip(dt, 2, 30) * nsig
plt_rng = [-clip, dt + clip]
trange = plt_rng
ntrials = 1

# Sample times
nevts = get_num_of_bg_events(t=t0, trange=trange, ntrials=ntrials,
                             pars=res.x)[0]
times = get_times_in_frame(t0, trange, nevts)

# Plot injected events from all trials
T = np.array([])
for ti in times:
    T = np.append(T, ti)  

h, b = np.histogram(T, bins=1081)
m = get_binmids([b])[0]
scale = np.diff(b) * secinday * ntrials
yerr = np.sqrt(h) / scale
h = h / scale

plt.errorbar(m, h, yerr=yerr, fmt=",")

# Plot normalized rate function to compare
t = np.linspace(start[0], stop[-1], 100)
r = rate_fun(t=t)
plt.plot(t, r, lw=2, zorder=5)
plt.axhline(res.x[3], 0, 1, color="C1", ls="--", label="", zorder=5)

plt.xlim(start[0], stop[-1])
plt.ylim(0, 0.009)
plt.tight_layout()

# plt.savefig("./data/figs/bg_events_time_sampled_wide.png", dpi=200)

plt.show()

# Testing for overlapping time windows snippet

In [ ]:
# Make overlapping time windows and raise error when we detect them
# Here window 4 overlaps with 3 and 5, which explains the matrix entries
dt0 = np.arange(1, 5 + 1) * 10
# Make this bigger or smaller to create overlap
overl = 10
dt1 = np.arange(1, 5 + 1) * 10 * overl
t = np.linspace(100, 1000, 5)

# Plot the windows
fig, ax = plt.subplots(1, 1)
for i, (ti, dti0, dti1) in enumerate(zip(t, dt0, dt1)):
    plt.axvspan(ti + dti0, ti + dti1, label="{:d}".format(i),
                color="C{}".format(i), alpha=.2)
    plt.axvline(ti + dti0, 0, 1, color="C{}".format(i), ls="--")
    plt.axvline(ti + dti1, 0, 1, color="C{}".format(i), ls="--")
plt.show()

# Test mutual exclusiveness of the windows
# larger edge must be y than lower edge and vice versa to be exclusive
exclusive = (((t + dt0)[:, None] >= t + dt1) |
             ((t + dt1)[:, None] <= t + dt0))
# Fix manually that time windows overlap with themselves of course
np.fill_diagonal(exclusive, True)

# If any entry is False, we have an overlapping window case
# Here it is (2, 3), (3, 2), (3, 4) and (4, 3)  as expected per construction
print(exclusive)

# Now raise errors
if np.any(exclusive == False):
    window_ids = [[x, y] for x, y in zip(*np.where(~exclusive))]
    raise ValueError("Overlapping time windows: {}.".format(", ".join(
        ["[{:d}, {:d}]".format(*c) for c in window_ids])) +
                     "\nThis is not supported yet.")

# Utils -- rotator

Compare to mhuber raw_flux and event selection 

**We need to change the kernel to Python 2 here, to load skylab**

No modifications have been done to skylab code.

The conversion from flux to fluence is simply the lack of livetime in the weights.
So we obtain skylab fluence by just repeating the steps from the code, but dropping the multiplication with the livetime.

The event selection is not affected by the livetime, so we don't have to modify the code itself.

In [ ]:
nsrc = 5
src_ra = np.linspace(0, 2. * np.pi, nsrc)
src_dec = np.deg2rad([-60., -30., 0, 30., 60.])
src_t = np.linspace(50000, 50300, nsrc)
dt0 = np.zeros(nsrc, dtype=np.float)
dt1 = np.zeros(nsrc, dtype=np.float) + 200.
w_theo = np.ones(nsrc)
src_names = ["ra", "dec", "t", "dt0", "dt1", "w_theo"]
srcs = np.core.records.fromarrays(np.vstack((src_ra, src_dec,src_t,
                                             dt0, dt1, w_theo)),
                                  names=src_names)

# Only set the src params we need here manually
sig_inj = SigInj.SignalInjector(gamma=2., mode="band",
                                inj_width=np.arcsin(0.1))

# Simulate skylab's band selection (debug flag only)
sig_inj._skylab_band = True

# Correct ow to ow_per_type
mc["ow"] = mc["ow"] / 0.5

# Different sample sizes
mc_dict = {0: mc, 1: mc[::10]}
livetime = {0: 340., 1: 34.}
sig_inj.fit(srcs, mc_dict, exp.dtype.names)

print("\nRaw Flux: {}".format(sig_inj._raw_flux))

# Undo ow to ow_per_type
mc["ow"] = mc["ow"] * 0.5

In [ ]:
print(sig_inj)

In [ ]:
import sys
sys.path.insert(1, "/Users/tmenne/icecube/software/skylab-mhuber")

from skylab.ps_injector import StackingPointSourceInjector as psinj
inj = psinj(gamma=2.)
inj.fill(src_dec, mc_dict, livetime)

ow = np.empty(0, dtype=np.float)
omega = (inj._omega / inj.w_theo)[inj.mc_arr["src_idx"]]
print("")
for enum in mc_dict.keys():
    idx = inj.mc_arr[inj.mc_arr["enum"] == enum]["idx"]
    _ow = (inj.mc[enum]["ow"] * inj.mc[enum]["trueE"]**(-inj.gamma))[idx]
    _ow /= omega[inj.mc_arr["enum"] == enum][idx]
    ow = np.append(ow, _ow)
    print("Raw Fluence at {} : {}".format(enum, np.sum(_ow)))

print("\nRaw Flux: {}".format(np.sum(ow)))

In [ ]:
# Are both raw_fluences and number of events equal?
print("Selected events equal : {}".format(len(sig_inj.mc_arr) ==
                                          len(inj.mc_arr)))
print("Raw fluences equal    : {}".format(np.allclose(sig_inj._raw_flux,
                                                      np.sum(ow))))

# Percentile with nzeros

In [ ]:
def _percentile_nzeros(vals, nzeros, q):
    """
    Returns the percentile q for a dataset with `vals` > 0 and `nzeros`
    entries that are zero.
    
    Parameters
    ----------
    vals : array-like
        Non-zero values.
    nzeros : int
        Number of zero trials.
    q : float
        Percentile in [0, 100].
    """
    q /= 100.
    nonzero = len(vals)
    ntot = nonzero + nzeros
    idx = int(q * ntot) - nzeros - 1
    vals = np.sort(vals)
        
    if idx < 0:
        return 0.
    else:
        return vals[idx]
    
if True:  # Test it
    nzeros = 100
    a = np.arange(10) + 1
    b = np.concatenate((np.zeros(nzeros), a))

    qs = np.arange(99) + 1

    for q in qs:
        print("- q   : ", q)
        print("  np  : ", np.percentile(b, q, interpolation="lower"))
        print("  own : ", _percentile_nzeros(a, nzeros, q))

# Interpolator options

skylab is linearly interpolating in logspcae.
Here we see what this mean (and also test the other settings for 1D).

The effect basically that we smooth the lower values and get more conservative to higher values.

In [ ]:
rndgen = np.random.RandomState(28756)

x = np.arange(10)
y = rndgen.randint(1, 1000, len(x))

x_ = np.linspace(x[0], x[-1], 1000)

# Linear interpol in logspace
interpol = sci.interp1d(x, np.log(y), kind="linear")
y_log = np.exp(interpol(x_))

# Normal space interpol
interpol = sci.interp1d(x, y, kind="linear")
y_lin = interpol(x_)
interpol = sci.interp1d(x, y, kind="quadratic")
y_quad = interpol(x_)
interpol = sci.interp1d(x, y, kind="cubic")
y_cube = interpol(x_)
interpol = sci.interp1d(x, y, kind="nearest")
y_near = interpol(x_)

plt.plot(x, y, color="k")
plt.plot(x_, y_log, ls="--", label="log-lin")
plt.plot(x_, y_lin, ls="--", label="lin")
plt.plot(x_, y_quad, ls="--", label="quad")
plt.plot(x_, y_cube, ls="--", label="cube")
plt.plot(x_, y_near, ls="--", label="near")

plt.xlabel("x")
plt.ylabel("y")
plt.legend(loc="best")

# plt.savefig("./data/figs/interpol_test_lin_log.png", dpi=200)
plt.show()


Do the same thing but now look at the interpolations in log space.
The log space linear interpolation is now working as the linear interpolation in normal space before.

In [ ]:
rndgen = np.random.RandomState(28756)

x = np.arange(10)
y = rndgen.randint(1, 1000, len(x))

x_ = np.linspace(x[0], x[-1], 1000)

# Linear interpol in logspace
interpol = sci.interp1d(x, np.log(y), kind="linear")
y_log = interpol(x_)

# Normal space interpol
interpol = sci.interp1d(x, y, kind="linear")
y_lin = np.log(interpol(x_))
interpol = sci.interp1d(x, y, kind="quadratic")
y_quad = np.log(interpol(x_))
interpol = sci.interp1d(x, y, kind="cubic")
y_cube = np.log(interpol(x_))
interpol = sci.interp1d(x, y, kind="nearest")
y_near = np.log(interpol(x_))

plt.plot(x, np.log(y), color="k")
plt.plot(x_, y_log, ls="--", label="log-lin")
plt.plot(x_, y_lin, ls="--", label="lin")
plt.plot(x_, y_quad, ls="--", label="quad")
plt.plot(x_, y_cube, ls="--", label="cube")
plt.plot(x_, y_near, ls="--", label="near")

plt.xlabel("x")
plt.ylabel("ln(y)")
plt.legend(loc="best")

# plt.savefig("./data/figs/interpol_test_lin_log_in_logspace.png", dpi=200)
plt.show()

# Time between run starts

Here we look at the time between run stop time and the start time of the next run to justify the handling of flaot errors in runtime binning.
Few events are outside any run but very close to a start or stop time, so they should be included in that run.
To account for that, we check if the events are within the bin edges +- eps and choose eps to be 1 second which is 1.15e-05 in MJD times.

Here we check if runs are usually further apart than 1 second so we introduce no double counting with the eps expansion.

**Result:** Normal runs seem to be started with no time loss. So we should not do the eps thing.
But then it's strange where those stray events come from, that are almost a second outside the nearest run...

In [ ]:
def create_goodrun_dict(runlist):
    if "runs" not in runlist.keys():
        raise ValueError("Runlist misses key 'runs' on top level")

    # This is a list of dicts (one dict per run)
    goodrun_list = runlist["runs"]

    # Convert the run list of dicts to a dict of arrays for easier handling
    goodrun_dict = dict(zip(goodrun_list[0].keys(),
                            zip(*[r.values() for r in goodrun_list])))
    for k in goodrun_dict.keys():
        goodrun_dict[k] = np.array(goodrun_dict[k])

    # Add times to MJD floats
    goodrun_dict["good_start_mjd"] = astrotime(
        goodrun_dict["good_tstart"], format="iso").mjd
    goodrun_dict["good_stop_mjd"] = astrotime(
        goodrun_dict["good_tstop"], format="iso").mjd

    # Add runtimes in MJD days
    goodrun_dict["runtime_days"] = (goodrun_dict["good_stop_mjd"] -
                                    goodrun_dict["good_start_mjd"])

    return goodrun_dict

In [ ]:
with open("data/runlists/ic86-i-goodrunlist.json", "r") as f:
    runlist = json.load(f)
    
rundict = create_goodrun_dict(runlist)

tstart = np.sort(rundict["good_start_mjd"])
tstop = np.sort(rundict["good_stop_mjd"])

fraction = []
dt = np.logspace(-2, 3, num=100)
for dti in dt:
    mask = []
    for t0, t1 in zip(tstart[1:], tstop[:-1]):
        mask.append(np.isclose(t0, t1, rtol=0., atol=dti / secinday))
    fraction.append(np.sum(mask) / (len(tstart) - 1.))

plt.plot(np.log10(dt), fraction, ls="", marker="o")
plt.xlabel("dt between runs in log10(sec)")
plt.ylabel("fraction of runs with |t1-t0| <= dt")
plt.ylim(0, 1.1)
plt.show()

# delta chi2 RV, implementing scipy.stats.rv_continuous

Compare vanilla chi2 with delta chi2 where `1-eta` is the zero fraction.

### Note

The survival function is `1-cdf`, but for the delta chi2 we define it to be 1 at zero because no events are lost.
If we stick to `1-cdf` it would be `eta` at zero, wrongly indicating, that events are lost.
On the other hand, the cdf is nonzero at zero, indicating the number of zero trials which are included in the PDF definition range.

In [ ]:
class delta_chi2_gen(rv_continuous):
    """
    Class for a probability denstiy function modelled by using a:math`\chi^2`
    distribution for :math:`x > 0` and a constant fraction :math:`1 - \eta`
    of zero trials for :math`x = 0` (like a delta peak at 0).

    Notes
    -----
    The probability density function for `delta_chi2` is:

    .. math::

      \text{PDF}(x|\text{df}, \eta) =
          \begin{cases}
              (1-\eta)                &\text{for } x=0 \\
              \eta\chi^2_\text{df}(x) &\text{for } x>0 \\
          \end{cases}

    `delta_chi2` takes ``df`` and ``eta``as a shape parameter, where ``df`` is
    the standard :math:`\chi^2_\text{df}` degrees of freedom parameter and
    ``1-eta`` is the fraction of the contribution of the delta function at zero.
    """
    def _rvs(self, df, eta):
        # Determine fraction of zeros by drawing from binomial with p=eta
        s = self._size if not len(self._size) == 0 else 1
        nzeros = self._random_state.binomial(n=s, p=1. - eta, size=None)
        # If None, len of size is 0 and single scalar rvs requested
        if len(self._size) == 0:
            if nzeros == 1:
                return 0.
            else:
                return self._random_state.chisquare(df, size=None)
        # If no zeros or no chi2 is drawn for this trial, only draw one type
        if nzeros == self._size:
            return np.zeros(nzeros, dtype=np.float)
        if nzeros == 0:
            return self._random_state.chisquare(df, size=self._size)
        # All other cases: Draw, concatenate and shuffle to simulate a per
        # random number Bernoulli process with p=eta
        out = np.r_[np.zeros(nzeros, dtype=np.float),
                    self._random_state.chisquare(df,
                                                 size=(self._size - nzeros))]
        self._random_state.shuffle(out)
        return out

    def _pdf(self, x, df, eta):
        x = np.asarray(x)
        return np.where(x > 0., eta * scs.chi2.pdf(x, df=df), 1. - eta)

    def _logpdf(self, x, df, eta):
        x = np.asarray(x)
        return np.where(x > 0., np.log(eta) + scs.chi2.logpdf(x, df=df),
                        np.log(1. - eta))

    def _cdf(self, x, df, eta):
        x = np.asarray(x)
        return np.where(x > 0., (1. - eta) + eta * scs.chi2.cdf(x, df=df),
                        (1. - eta))

    def _logcdf(self, x, df, eta):
        x = np.asarray(x)
        return np.where(x > 0., np.log(1 - eta + eta * scs.chi2.cdf(x, df)),
                        np.log(1. - eta))

    def _sf(self, x, df, eta):
        # Note: Define sf(0)=0 and not sf(0)=(1-eta) because 0 is inclusive
        x = np.asarray(x)
        return np.where(x > 0., eta * scs.chi2.sf(x, df), 1.)

    def _logsf(self, x, df, eta):
        # Note: Define sf(0)=0 and not sf(0)=(1-eta) because 0 is inclusive
        x = np.asarray(x)
        return np.where(x > 0., np.log(eta) + scs.chi2.logsf(x, df), 0.)

    def _ppf(self, p, df, eta):
        p = np.asarray(p)
        return np.where(p > (1. - eta), scs.chi2.ppf(1 + (p - 1) / eta, df), 0.)

    def _isf(self, p, df, eta):
        return np.where(p < eta, scs.chi2.isf(p / eta, df), 0.)


delta_chi2 = delta_chi2_gen(name="delta_chi2")

In [ ]:
x = np.linspace(0, 2, 200)
p = np.linspace(0, 1, 200)
eta = 0.7
df = 1

chi2 = scs.chi2(df=df)
dchi2 = delta_chi2(df=df, eta=eta)

**PDF**

In [ ]:
fig, (axl, axr) = plt.subplots(1, 2, figsize=(14, 6));

axl.plot(x, chi2.pdf(x), lw=3)
axl.set_title(r"Vanilla $\chi^2_{}$".format(df))

axr.plot(x, dchi2.pdf(x), lw=3, c="C3")
axr.set_title(r"$(1-{:.1f}) + {:.1f}\times \chi^2_{}$".format(eta, eta, df))

for axi in (axl, axr):
    axi.axhline(1. - eta, 0, 1, ls="--", color="k", label=r"$1-\eta$")
    axi.set_xlabel("x")
    axi.set_ylabel("PDF")
    axi.set_xlim(x[0], x[-1])
    axi.set_ylim(0, 4)
    axi.grid(ls="--")
    axi.legend(loc="upper right")
plt.tight_layout()
plt.show()

**ln(PDF)**

In [ ]:
# Test if exp(logPDF) == PDF
print("exp(logPDF) is PDF: ", np.allclose(np.exp(dchi2.logpdf(x)),
                                          dchi2.pdf(x)))

In [ ]:
fig, (axl, axr) = plt.subplots(1, 2, figsize=(14, 6));

axl.plot(x, chi2.logpdf(x), lw=3)
axl.set_title(r"Vanilla $\chi^2_{}$".format(df))

axr.plot(x, dchi2.logpdf(x), lw=3, c="C3")
axr.set_title(r"$(1-{:.1f}) + {:.1f}\times \chi^2_{}$".format(eta, eta, df))

for axi in (axl, axr):
    axi.axhline(np.log(1. - eta), 0, 1, ls="--", color="k", label=r"$1-\eta$")
    axi.set_xlabel("x")
    axi.set_ylabel("ln(PDF)")
    axi.set_xlim(x[0], x[-1])
    axi.set_ylim(-2, 2)
    axi.grid(ls="--")
    axi.legend(loc="upper right")
plt.tight_layout()
plt.show()

**CDF**

In [ ]:
fig, (axl, axr) = plt.subplots(1, 2, figsize=(14, 6));

axl.plot(x, chi2.cdf(x), lw=3)
axl.set_title(r"Vanilla $\chi^2_{}$".format(df))

axr.plot(x, dchi2.cdf(x), lw=3, c="C3")
axr.set_title(r"$(1-{:.1f}) + {:.1f}\times \chi^2_{}$".format(eta, eta, df))

for axi in (axl, axr):
    axi.axhline(1. - eta, 0, 1, ls="--", color="k", label=r"$1-\eta$")
    axi.set_xlabel("x")
    axi.set_ylabel("CDF")
    axi.set_xlim(x[0], x[-1])
    axi.set_ylim(0, 1)
    axi.grid(ls="--")
    axi.legend(loc="upper right")
plt.tight_layout()
plt.show()

**ln(cdf)**

In [ ]:
# Test if exp(logCDF) == CDF
print("exp(logCDF) is CDF: ", np.allclose(np.exp(dchi2.logcdf(x)),
                                          dchi2.cdf(x)))

In [ ]:
fig, (axl, axr) = plt.subplots(1, 2, figsize=(14, 6));

axl.plot(x, chi2.logcdf(x), lw=3)
axl.set_title(r"Vanilla $\chi^2_{}$".format(df))

axr.plot(x, dchi2.logcdf(x), lw=3, c="C3")
axr.set_title(r"$(1-{:.1f}) + {:.1f}\times \chi^2_{}$".format(eta, eta, df))

for axi in (axl, axr):
    axi.axhline(np.log(1. - eta), 0, 1, ls="--", color="k", label=r"$1-\eta$")
    axi.set_xlabel("x")
    axi.set_ylabel("ln(CDF)")
    axi.set_xlim(x[0], x[-1])
    axi.set_ylim(-2, 0)
    axi.grid(ls="--")
    axi.legend(loc="upper right")
plt.tight_layout()
plt.show()

**PPF**

In [ ]:
fig, (axl, axr) = plt.subplots(1, 2, figsize=(14, 6));

axl.plot(p, chi2.ppf(p), lw=3)
axl.set_title(r"Vanilla $\chi^2_{}$".format(df))

axr.plot(p, dchi2.ppf(p), lw=3, c="C3")
axr.set_title(r"$(1-{:.1f}) + {:.1f}\times \chi^2_{}$".format(eta, eta, df))

# Overlay rotated CDF, should be the same function (graphical inversion)
axr.plot(dchi2.cdf(x), x, lw=2, ls="--", c="k", label="inverted CDF")

for axi in (axl, axr):
    axi.axvline(1. - eta, 0, 1, ls="--", color="k", label=r"$1-\eta$")
    axi.set_xlabel("p")
    axi.set_ylabel("PPF")
    axi.set_xlim(p[0], p[-1])
    axi.set_ylim(x[0], x[-1])
    axi.grid(ls="--")
    axi.legend(loc="upper left")
plt.tight_layout()
plt.show()

**Survival function, SF**

In [ ]:
fig, (axl, axr) = plt.subplots(1, 2, figsize=(14, 6));

axl.plot(x, chi2.sf(x), lw=3)
axl.set_title(r"Vanilla $\chi^2_{}$".format(df))

axr.plot(x, dchi2.sf(x), lw=3, c="C3")
axr.set_title(r"$(1-{:.1f}) + {:.1f}\times \chi^2_{}$".format(eta, eta, df))

# Overlay 1-CDF, should be the same function, except for x=0
axr.plot(x, 1. - dchi2.cdf(x), lw=2, ls="--", c="k", label="1-CDF")

for axi in (axl, axr):
    axi.set_xlabel("x")
    axi.set_ylabel("CDF")
    axi.set_xlim(x[0], x[-1])
    axi.set_ylim(0, 1)
    axi.grid(ls="--")
axr.legend(loc="upper right")
plt.tight_layout()
plt.show()

**ln(SF)**

In [ ]:
# Test if exp(logCDF) == CDF
print("exp(logSF) is SF: ", np.allclose(np.exp(dchi2.logsf(x)),
                                          dchi2.sf(x)))

In [ ]:
fig, (axl, axr) = plt.subplots(1, 2, figsize=(14, 6));

axl.plot(x, chi2.logsf(x), lw=3)
axl.set_title(r"Vanilla $\chi^2_{}$".format(df))

axr.plot(x, dchi2.logsf(x), lw=3, c="C3")
axr.set_title(r"$(1-{:.1f}) + {:.1f}\times \chi^2_{}$".format(eta, eta, df))

for axi in (axl, axr):
    axi.set_xlabel("x")
    axi.set_ylabel("ln(CDF)")
    axi.set_xlim(x[0], x[-1])
    axi.set_ylim(-2, 0)
    axi.grid(ls="--")
plt.tight_layout()
plt.show()

**inverse SF, ISF**

In [ ]:
fig, (axl, axr) = plt.subplots(1, 2, figsize=(14, 6));

axl.plot(p, chi2.isf(p), lw=3)
axl.set_title(r"Vanilla $\chi^2_{}$".format(df))

axr.plot(p, dchi2.isf(p), lw=3, c="C3")
axr.set_title(r"$(1-{:.1f}) + {:.1f}\times \chi^2_{}$".format(eta, eta, df))

# Overlay rotated SF, should be the same function (graphical inversion)
axr.plot(dchi2.sf(x), x, lw=2, ls="--", c="k", label="inverted SF")

for axi in (axl, axr):
    axi.set_xlabel("p")
    axi.set_ylabel("PPF")
    axi.set_xlim(p[0], p[-1])
    axi.set_ylim(x[0], x[-1])
    axi.grid(ls="--")
axr.legend(loc="upper right")
plt.tight_layout()
plt.show()

**Random numbers**

In [ ]:
fig, (axl, axr) = plt.subplots(1, 2, figsize=(14, 6));

# Draw RVS, seperate almost pure zero bin
bins = np.arange(x[0], x[-1], 0.05)
nrnd = 10000
chi2rvs = chi2.rvs(size=nrnd)
dchi2rvs = dchi2.rvs(size=nrnd)

axl.hist(chi2rvs, bins=bins, normed=True, color="C7", log=True)
axl.plot(x, chi2.pdf(x), lw=3)
axl.set_title(r"Vanilla $\chi^2_{}$".format(df))

axr.hist(dchi2rvs, bins=bins, normed=True, color="C7", log=True)
axr.plot(x, dchi2.pdf(x), lw=3, c="C3")
axr.set_title(r"$(1-{:.1f}) + {:.1f}\times \chi^2_{}$".format(eta, eta, df))

for axi in (axl, axr):
    axi.set_xlabel("x")
    axi.set_ylabel("ln(PDF)")
    axi.set_xlim(x[0], x[-1])
    axi.grid(ls="--")
plt.tight_layout()
plt.show()

**The old comparison, also showing that skylabs logCDF is wrong**

In [ ]:
df = 1
eta = 0.7
x = np.linspace(0, 10, 200)

# lin vanilla
cdf = scs.chi2.cdf(x, df)
sf = scs.chi2.sf(x, df)
sf_ = 1 - cdf
# delta chi2
cdf_eta = np.where(x > 0, 1 - eta + eta * scs.chi2.cdf(x, df), 1 - eta)
sf_eta = np.where(x > 0, eta * scs.chi2.sf(x, df), 1.)
sf_eta_ = 1 - cdf_eta  # Incorrect here, sf(0) = 1

#log vanilla
logcdf = scs.chi2.logcdf(x, df)
logsf = scs.chi2.logsf(x, df)
logsf_ = np.log(1 - cdf)
# delta chi2
logcdf_eta = np.where(x > 0., np.log(1 - eta + eta * scs.chi2.cdf(x, df)),
                      np.log(1. - eta))
logcdf_eta_2 = np.log(cdf_eta)
logsf_eta =  np.where(x > 0, np.log(eta) + scs.chi2.logsf(x, df), 0.)
logsf_eta_ = np.log(1 - cdf_eta)  # Incorrect here, sf(0) = 1

#skylab
cdf_eta_skylab = (1. - eta) + eta * scs.chi2.cdf(x, df)
logcdf_eta_skylab = np.log(1. - eta) + np.log(eta) + scs.chi2.logcdf(x, df)

# Check exp(log) = lin
print("CDF: ", np.allclose(np.exp(logcdf_eta), cdf_eta))
print("SF: ", np.allclose(np.exp(logsf_eta), sf_eta))
print("CDF skylab: ", np.allclose(np.exp(logcdf_eta_skylab), cdf_eta_skylab))

# lin
plt.plot(x, cdf)
plt.plot(x, sf)
plt.plot(x, sf_, ls="--")
plt.axhline(1,0,1,ls="--", c="k")
plt.show()

plt.plot(x, cdf_eta)
plt.plot(x, sf_eta)
plt.plot(x, sf_eta_, ls="--")
plt.plot(x, cdf_eta_skylab, ls="--")
plt.axhline(1,0,1,ls="--", c="k")
plt.show()

# log
plt.plot(x, logcdf)
plt.plot(x, logsf)
plt.plot(x, logsf_, ls="--")
plt.axhline(0,0,1,ls="--", c="k")
plt.show()

plt.plot(x, logcdf_eta)
plt.plot(x, logcdf_eta_2, c="k", ls="--")
plt.plot(x, logsf_eta)
plt.plot(x, logsf_eta_, ls="--")
plt.plot(x, logcdf_eta_skylab, ls="--")
plt.axhline(0,0,1,ls="--", c="k")
plt.show()

# Placeholder for new test